<a href="https://colab.research.google.com/github/vilcagamarracf/Playing-with-Python/blob/main/Crop_classification_EE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***Important Note: Access to [Google Earth Engine](https://earthengine.google.com/) is currently only available to registered users. The [Google Earth Engine API](https://github.com/google/earthengine-api) is in active development, and users should expect the API to change. When (not if) API changes occur, applications that use the API will likely need to be updated.***


<center>
<h1>Google Earth Engine with Python </h1>
<h2> Crop Classification using the GEE Python API, Valle de Chincha, Ica. </h2>
</center>

***Information about this project***
```
FILE          : 00_Tesis_Cesar_Vilca.ipynb 
AUTHOR        : Cesar Francisco Vilca Gamarra 
EMAIL         : vilcagamarracf@gmail.com 
ORGANIZATION  : UNALM 
CREATION DATE : 15 June 2020 
DEPENDENCIES  : earthengine-api, os, pandas, urllib, plotly, numpy, folium 
PURPOSE       : Download Sentinel Products 
```
<center>
<img src= 'https://miro.medium.com/max/1350/1*bOokbEeXpF1Z4gd_BpL93w.jpeg' width=50%/>
</center>


In [ ]:
#@markdown Banner
%%html
<marquee style='width: 30%; color: blue;'><b>¡Tesis en proceso! </b></marquee>

# *1. Introducción*

"*Aquí va una introducción interesante*"

Hablar de:
- Situación del país
- ¿Es necesario realizar esta investigación? (clasificar cultivos)
- Aprendizaje automático (*Machine Learning* en ingles)
- Su aplicación en la Agricultura


# *2. Metodología*

La presente investigación se desarrollará de la siguiente manera:

1. **Fase: Earth Engine**
  - **Earth Engine**: Obtención y pre-procesamiento (control de calidad) de imágenes satelitáles   
3. **Fase: ML-Random Forest**
  - Elaboración del modelo (Algoritmo de clasificación Random Forest)
  - Procesamiento de imágenes con el modelo (Entrenamiento)
  - Validación del modelo (con datos de campo)

# *3. Desarrollo*



## Fase 1: Earth Engine






**Objetivo**:
- Obtener imágenes (de EE) del área de estudio para su procesamiento y posterior ingesta al modelo (clasificación supervisada mediante el método de ensamble Random Forest).

Pasos:

1. Creación de cuenta en EE
2. EE en Colab (como IDE de python de manera online)
3. Obtención de imágenes satelitales [Sentinel-2](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR#description) mediante EE en Colab ([EE Datasets](https://developers.google.com/earth-engine/datasets/))
4. Tratamiento de imágenes (máscara de nubes - [Sentinel-2 Cloud Probability](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_CLOUD_PROBABILITY)
)

### Trabajo Previo en GEE

1. GEE Python API en Colab
2. Region of Interest (ROI): Chincha
3. Visualización rápida de imágenes (Mediana)
4. Visualización de propiedades
5. Visualización de mejor imagen

#### ***1. GEE Python API en Colab***

Revisar la versión actual de [Google Earth Engine API](https://github.com/google/earthengine-api)

In [ ]:
# @markdown Ver upgrade de packages: [foro](https://stackoverflow.com/questions/5183672/how-do-i-update-a-python-package)
!pip list --outdated

# https://medium.com/python-in-plain-english/understand-python-import-module-and-package-c5b4289bbf60

In [ ]:
#Revisar la versión actual de Google Earth Engine API
# https://github.com/google/earthengine-api
import ee
print('GEE Version:',ee.__version__)

GEE Version: 0.1.258


In [ ]:
# Para actualizar la versión de google earth engine. 
# 29 de Octubre, 2020: GEE Version: 0.1.240
#!pip install folium --upgrade 

!pip install earthengine-api --upgrade 

In [ ]:
#@markdown *Credenciales Google Earth Engine*
import os 
credential = '{"refresh_token":"1//09lhqedCNpNDsCgYIARAAGAkSNwF-L9IraEP8FyJma46WSiEHhOMsZqSoTPJsi3Lar0C4nZUwj2QCZ_i66-McVzy4NmMalQL17KM"}'
credential_file_path = os.path.expanduser("~/.config/earthengine/")
os.makedirs(credential_file_path,exist_ok=True)
with open(credential_file_path + 'credentials', 'w') as file:
    file.write(credential)
import ee
ee.Initialize()
print('Ya estás logeado.')
print('GEE Version:',ee.__version__)

Ya estás logeado.
GEE Version: 0.1.260


In [ ]:
# @markdown Importar:
# @markdown - `mapdisplay`: Variable global: `mapViz`
# @markdown - `ee_get_dates`
# @markdown - `conv_fecha`
# @markdown - `ver_rangos`
# @markdown - `ROI CHINCHA`
# @markdown - `create_dateranges`


# mapdisplay
import folium 
def mapdisplay(center, dicc, Tiles="OpensTreetMap",zoom_start=10):
    '''
    Crea mapas interactivos usando la librería `folium`

    - `center`: Center of the map (Latitude and Longitude) (`list`)
    - `dicc`: Earth Engine Geometries or Tiles dictionary (`dict`)
    - `Tiles=`: 
    Mapbox Bright 
    Mapbox Control Room
    Stamen Terrain
    Stamen Toner
    stamenwatercolor
    cartodbpositron
    - `zoom_start=`: Initial zoom level for the map (`int`)

    Return: A folium.Map object
    '''
    # Centrado de imagen (El [::-1] lee inversamente la lista)
    center = center[::-1]

    # Agregando parámetros de visualización
    global mapViz
    mapViz = folium.Map(location=center,tiles=Tiles, zoom_start=zoom_start, control_scale=True)

    # Seteando la visualización de Images
    for k,v in dicc.items():
      if ee.image.Image in [type(x) for x in v.values()]:
        folium.TileLayer(tiles = v["tile_fetcher"].url_format,
                         attr  = 'Google Earth Engine',
                         overlay =True,
                         name  = k).add_to(mapViz)
      else:
        folium.GeoJson(data = v,
                       name = k).add_to(mapViz)
    # Agregar: Minimapa
    from folium.plugins import MiniMap
    minimap = MiniMap(
        tile_layer="OpensTreetMap", 
        position="bottomright", 
        zoom_level_fixed=7)
    mapViz.add_child(minimap) # https://nbviewer.jupyter.org/github/python-visualization/folium/blob/master/examples/MiniMap.ipynb

    # Fullscreen (no funciona en colab)
    # plugins.Fullscreen().add_to(mapViz)

    # Agregando el control de layers
    mapViz.add_child(folium.LayerControl())
    return mapViz
print('mapdisplay(center, dicc, Tiles="OpensTreetMap",zoom_start=10) listo!')

# ee_get_dates ----------------------------------------
from datetime import datetime as dt
def ee_get_dates(ic):
  """
  Permite obtener información de la Fecha y Hora de toma de imágen por el satélite.
  - ic: `ee.ImageCollection()`
  """
  dates = ic.aggregate_array('system:time_start').getInfo()
  anonym = lambda x: dt.utcfromtimestamp(x/1000).strftime('%Y-%m-%d %H:%M:%S')
  return list(map(anonym, dates))
print('ee_get_dates(ic) listo!')

# conv_fecha ----------------------------------------
def conv_fecha(img, getInfo=None):
  '''
  Convierte el formato de UNIX Time al formato %Y-%m-%d %H:%M:%S
  img: ee.Image
  getInfo = True : cuando se tiene ya el .getInfo() 
  '''
  from datetime import datetime as dt
  if getInfo is None:
    return dt.utcfromtimestamp(img.getInfo()['properties']["system:time_start"]/1000).strftime('%Y-%m-%d %H:%M:%S')
  if getInfo==True:
    return dt.utcfromtimestamp(img['properties']["system:time_start"]/1000).strftime('%Y-%m-%d %H:%M:%S')
print('conv_fecha(img, getInfo=None) listo!')

# ROI CHINCHA ----------------------------------------

area_total = ee.FeatureCollection('users/CesarVilca/area_total')
roi_clip = area_total.geometry().bounds()

centroide = [-76.0908, -13.4589]
print('roi, roi_clip y centroide listos!')

# ver_rangos

Start_date = '2012-01-01'
End_date   = '2021-01-31'
range_date = [Start_date, End_date]

def ver_rangos(img_col, range_date= [Start_date, End_date]):
  """
  Devuelve los rangos de fechas existente en un rango inicial escrito manualmente como input.
  Pasos:
  1. `Reducer.minMax()` devuelve un valor min y max
  2. `icol.reduceColumns()` devuelve un diccionario con el min-max
  more info: https://developers.google.com/earth-engine/apidocs/ee-imagecollection-reducecolumns?hl=en
  """
  
  rango = img_col.reduceColumns(ee.Reducer.minMax(), ["system:time_start"]) # Retorna un ee.dictionary: {'max': 1608132402761, 'min': 1545060401000}

  # Obtención de la fecha min y max del image collection en formato ISO standard 8601
  # Javascript trabaja las fechas con milisegundos (se deja así)
  # Python     trabaja las fechas con segundos (por eso /1000)
  init_date = ee.Date(rango.get('min')).getInfo()['value']/1000.
  last_date = ee.Date(rango.get('max')).getInfo()['value']/1000.

  # Dar formato a las fechas  
  from datetime import datetime as dt
  init_date_f = dt.utcfromtimestamp(init_date).strftime('%Y-%m-%d') # %H:%M:%S
  last_date_f = dt.utcfromtimestamp(last_date).strftime('%Y-%m-%d') # %H:%M:%S
  
  # Obtenga el rango de fechas de las imágenes en la colección.
  range_date = [Start_date, End_date]
  print('Total imágenes: {}'.format(img_col.size().getInfo()))
  print('Rango Temporal Ingresado : {} - {}'.format(range_date[0], range_date[1]))
  print('Rango Temporal Real      : {} - {}'.format(init_date_f, last_date_f))
print('ver_rangos(icol, range_date= [Start_date, End_date]) listo!')

# create_dateranges ----------------------------------------
def create_dateranges(init_date, time, range_1, range_2):
  '''
  init_date: '20XX' for monthly dates, '20XX-XX' for daily dates(string)
  time: 'year', 'month', 'day' (string)
  range_1: 1, ... 12 for monthly values (int)
  range_2: 1, ... 13 for monthly values (int)

  Returns a date ranges in a list form.
  '''
  ranges = []
  def create_dates(date, plus_time):
     ''' 
     init_date: string values only, example:  '2013-1'
     time: 'year', 'month', 'day'
  
     more info: https://developers.google.com/earth-engine/apidocs/ee-date-getrange
     '''
     var_time = ee.Date(date).getRange(plus_time).getInfo().get('dates')
  
     from datetime import datetime as dt #-%d
     Start_Date = dt.utcfromtimestamp(var_time[0]/1000).strftime('%Y-%m')
     End_Date   = dt.utcfromtimestamp(var_time[1]/1000).strftime('%Y-%m')
     return  [Start_Date, End_Date, i-1]
  
  for i in range(range_1, range_2+1):
    ranges.append(create_dates(init_date+'-'+str(i), time))
  return ranges
print('create_dateranges(init_date, time, range_1, range_2) listo!')

mapdisplay(center, dicc, Tiles="OpensTreetMap",zoom_start=10) listo!
ee_get_dates(ic) listo!
conv_fecha(img, getInfo=None) listo!
roi, roi_clip y centroide listos!
ver_rangos(icol, range_date= [Start_date, End_date]) listo!
create_dateranges(init_date, time, range_1, range_2) listo!


#### ***2. Region of Interest (ROI): Chincha***

In [ ]:
area_total = ee.FeatureCollection('users/CesarVilca/area_total')
roi_clip = area_total.geometry().bounds()

centroide = [-76.0908, -13.4589]

# Visualización
# dicc = {'area total': area_total.getInfo(), 
#         'roi_clip'  : roi_clip.getInfo()}

# mapdisplay(centroide, dicc, zoom_start=12)

#### ***3. Visualización rápida de imágenes (Mediana)***

Source: [Exploring Satellite Imagery — Part II - Girija Shankar Behera](https://medium.com/swlh/exploring-satellite-imagery-part-ii-44bd2a5ee85c)

> Sentinel-2A launched on 23June, 2015 and Sentinel-2B launched on 7March, 2017 under the Copernicus programme as Earth Observation satellites. The two satellites also known as twin satellites, form a constellation, phasing at 180° to each other. The earlier one alone is able to complete a full cycle of earth in about 10days, with the second one coming up, they together take 5 days for the same.


- **OJO**: La mediana (`.median()`) produce mejores visualizaciones a comparación del promedio `.mean()`.

 1. Identificar el producto y fecha de interés disponible:

Satellite | Data Availability | For the study area
:---: | :---: | :---:
Landsat 7 SR | 1999-01-01 – Present | ---
Landsat 8 SR | 2013-04-11 – Present | ---
Sentinel-2   | 2015-06-23 – Present | 2015-11-28 – Present
Sentinel-2 SR| 2017-03-28 – Present | 2018-12-17 – Present

 2. Selección del producto:
 - USGS Landsat 7 Surface Reflectance Tier 1: `ee.ImageCollection("LANDSAT/LE07/C01/T1_SR")`
 - USGS Landsat 8 Surface Reflectance Tier 1: `ee.ImageCollection("LANDSAT/LC08/C01/T1_SR")`
 - Sentinel-2 MSI: MultiSpectral Instrument, Level-1C: `ee.ImageCollection("COPERNICUS/S2")`
 - Sentinel-2 MSI: MultiSpectral Instrument, Level-2A: `ee.ImageCollection("COPERNICUS/S2_SR")`



In [ ]:
#@title Visualización de Imágenes Satelitales

#-----------------------------------------
#    Filtrado de ee.ImageCollection()
#-----------------------------------------

#@markdown 1. Selección del producto:
Product_name = 'COPERNICUS/S2_SR'  #@param ["COPERNICUS/S2", "COPERNICUS/S2_SR"]

#@markdown 2. Seleccionamos las fechas de interés:
Start_date = '2012-12-17'  #@param {type: "date"}
End_date   = '2021-02-12'  #@param {type: "date"}

#@markdown 3. Porcentaje de Nubosidad:
Cloud_Pixel_Percentage =    100#@param {type:"number"}

#  Parametros globales
range_date = [Start_date, End_date]

#@markdown 4. Variable del ImageColecction: **`img_col`**
# ee.ImageCollection filtrado de fechas, area y % de nubes
img_col = ee.ImageCollection(Product_name)\
         .filterDate(range_date[0], range_date[1])\
         .filterBounds(ee.Geometry.Point(-76.13, -13.48))\
         .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', Cloud_Pixel_Percentage)) 

#-----------------------------------------
#                  Fechas
#-----------------------------------------
# Ver rangos temporales reales e introducidos manualmente + cantidad de imgs
ver_rangos(img_col, range_date=range_date)

#-----------------------------------------
#              Visualización 
#-----------------------------------------
#@markdown 5. Visualización de imagen en RGB (Median) 
# Retorna: ee.Image
img_col_clip = img_col.select(['B4','B3','B2'])\
                        .median()\
                        .divide(10000)\
                        .clip(roi_clip) # válido solo en ee.Image

# Visualización de la imagen en color natural
bands  = ['B4','B3','B2'] # Natural Color
mydicc = {
    'Natural Color': img_col_clip.getMapId({'min':0,'max':0.3,'bands':bands}),
    'Region of interest (ROI)': area_total.getInfo()}

mapdisplay(centroide, mydicc,zoom_start=12)

Total imágenes: 156
Rango Temporal Ingresado : 2012-12-17 - 2021-02-12
Rango Temporal Real      : 2018-12-17 - 2021-02-09


#### **4. Visualización de propiedades**

In [ ]:
# @markdown ee.ImageCollection de prueba
Start_date = '2018-12-11'
End_date   = '2020-01-31'
range_date = [Start_date, End_date]

img_col = ee.ImageCollection('COPERNICUS/S2_SR')\
            .filterDate(Start_date, End_date)\
            .filterBounds(ee.Geometry.Point(-74.904844,-12.401923))\
            .sort('CLOUDY_PIXEL_PERCENTAGE')

In [ ]:
# Leer diccionarios con .keys()
img_col.getInfo().keys() # Output: dict_keys(['type', 'bands', 'id', 'version', 'properties', 'features'])

# Leer values de un key (diccionario -> key:value)
img_col.getInfo()['properties'] # Probar con la primera imagen: .first()

# Cantidad de imagenes en ee.ImageCollection
img_col.size().getInfo() # Output: 82

In [ ]:
# Ver bandas de una imagen
img_col.first().bandNames().getInfo()   # Output: ['B4', 'B3', 'B2']

# Ver ID:
img_col.first().get('system:index').getInfo()       # Output: 20190730T151711_20190730T151807_T18LWM

# Visualizar propiedades de una imagen
img_prueba1 = ee.Image('COPERNICUS/S2_SR/20190730T151711_20190730T151807_T18LWM')
img_prueba1.getInfo()['properties']


> Propiedades de ImageCollection: `ee.ImageCollection.aggregate_stats('propiedad')`

- [Sentinel-2 MSI: MultiSpectral Instrument, Level-2A](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR#image-properties)
- Ojo: Las propiedades de tipo DOUBLE si abren, los de tipo STRING no


In [ ]:
# Listar propiedades
for i, key in enumerate(img_col.first().getInfo()['properties'].keys(), 1):
  print(i, key) # 59: CLOUDY_PIXEL_PERCENTAGE

1 DATATAKE_IDENTIFIER
2 AOT_RETRIEVAL_ACCURACY
3 SPACECRAFT_NAME
4 SATURATED_DEFECTIVE_PIXEL_PERCENTAGE
5 MEAN_INCIDENCE_AZIMUTH_ANGLE_B8A
6 CLOUD_SHADOW_PERCENTAGE
7 MEAN_SOLAR_AZIMUTH_ANGLE
8 system:footprint
9 VEGETATION_PERCENTAGE
10 SOLAR_IRRADIANCE_B12
11 SOLAR_IRRADIANCE_B10
12 SENSOR_QUALITY
13 SOLAR_IRRADIANCE_B11
14 GENERATION_TIME
15 SOLAR_IRRADIANCE_B8A
16 FORMAT_CORRECTNESS
17 CLOUD_COVERAGE_ASSESSMENT
18 THIN_CIRRUS_PERCENTAGE
19 system:time_end
20 WATER_VAPOUR_RETRIEVAL_ACCURACY
21 system:time_start
22 DATASTRIP_ID
23 PROCESSING_BASELINE
24 SENSING_ORBIT_NUMBER
25 NODATA_PIXEL_PERCENTAGE
26 SENSING_ORBIT_DIRECTION
27 GENERAL_QUALITY
28 GRANULE_ID
29 REFLECTANCE_CONVERSION_CORRECTION
30 MEDIUM_PROBA_CLOUDS_PERCENTAGE
31 MEAN_INCIDENCE_AZIMUTH_ANGLE_B8
32 DATATAKE_TYPE
33 MEAN_INCIDENCE_AZIMUTH_ANGLE_B9
34 MEAN_INCIDENCE_AZIMUTH_ANGLE_B6
35 MEAN_INCIDENCE_AZIMUTH_ANGLE_B7
36 MEAN_INCIDENCE_AZIMUTH_ANGLE_B4
37 MEAN_INCIDENCE_ZENITH_ANGLE_B1
38 NOT_VEGETATED_PERCENTAGE
39 ME

In [ ]:
# Estadísticas de un ee.ImageCollection
img_col.aggregate_stats('CLOUDY_PIXEL_PERCENTAGE').getInfo() # Notese la imagen con mínimo porcentaje de nubes

# Ejemplo: Todos los porcentajes de nubosidad en cada imagen mediante un histograma
img_col.aggregate_histogram('CLOUDY_PIXEL_PERCENTAGE').getInfo()

In [ ]:
# Comparando propiedades: Con y sin clip (¿Cambian las propiedades o no?)
img_prueba1 = ee.Image('COPERNICUS/S2_SR/20190730T151711_20190730T151807_T18LWM')
img_prueba1.getInfo()['properties']

img_prueba2 = ee.Image('COPERNICUS/S2_SR/20190730T151711_20190730T151807_T18LWM').clip(roi_clip)
img_prueba2.getInfo()['properties'] # Ver en otra celda

*Evaluando el primer mes de 2019 (`s_19_01_imgc`)*

In [ ]:
# ImageCollection para el mes de enero 2019
img_s2_enero = ee.ImageCollection('COPERNICUS/S2_SR')\
                 .filterDate('2019-01', '2019-02')\
                 .filterBounds(roi)

# guardar diccionario de metadatos del image collection
dic0 = img_s2_enero.getInfo() 

# Imprimir las keys del diccionario (metadatos del imagecollection)
print(dic0.keys()) # dict_keys(['type', 'bands', 'id', 'version', 'properties', 'features']) 

# Histograma de la propiedad 'CLOUDY_PIXEL_PERCENTAGE'
img_s2_enero.aggregate_histogram('CLOUDY_PIXEL_PERCENTAGE').getInfo() # No es en orden de fechas, sino numérico

# Ver fechas
ee_get_dates(img_s2_enero) # Función
conv_fecha(img_s2_enero.first())

{'11.442228': 1,
 '37.127514': 1,
 '43.483208': 1,
 '48.092331': 1,
 '52.611271': 1,
 '58.199064': 1,
 '63.706253': 1}

In [ ]:
for i in range(7): # [0,7> dependiendo de la cantidad de imagenes, si no esta en el rango -> IndexError: list index out of range
  print(str(dic0['features'][i]['properties']['CLOUDY_PIXEL_PERCENTAGE'])+' %',
        ':' ,
        dt.utcfromtimestamp(dic0['features'][i]['properties']['GENERATION_TIME']/1000).strftime('%Y-%m-%d %H:%M:%S')) # system:time_start vs GENERATION_TIME

48.092331 % : 2019-01-01 18:56:59
58.199064 % : 2019-01-06 20:46:37
43.483208 % : 2019-01-11 18:58:38
37.127514 % : 2019-01-16 19:19:16
63.706253 % : 2019-01-21 18:58:40
11.442228 % : 2019-01-26 19:18:31
52.611271 % : 2019-01-31 19:00:34


In [ ]:
# .sort para odernar ascendente o descendente según la propiedad 
#dic1 = s_19_01_imgc.sort('CLOUDY_PIXEL_PERCENTAGE', True).getInfo() # getInfo() retorna diccionarios

# No hacer
#print(dic0.items())

In [ ]:
dic0['features'][0].keys()

dict_keys(['type', 'bands', 'id', 'version', 'properties'])

In [ ]:
img_s2_enero.aggregate_stats('CLOUDY_PIXEL_PERCENTAGE').getInfo()

{'max': 63.706253,
 'mean': 44.951695571428566,
 'min': 11.442228,
 'sample_sd': 17.233772904754087,
 'sample_var': 297.0029285326361,
 'sum': 314.66186899999997,
 'sum_sq': 15926.60211442055,
 'total_count': 7,
 'total_sd': 15.955373350136922,
 'total_var': 254.5739387422595,
 'valid_count': 7,
 'weight_sum': 7,
 'weighted_sum': 314.66186899999997}

In [ ]:
img_s2_enero.propertyNames().getInfo()

In [ ]:
# Obtener la mejor imágen (respecto al porcentaje de nubes)
image = ee.Image(img_s2_enero.sort('CLOUDY_PIXEL_PERCENTAGE').first()) # no es necesario colocar ,opt_ascending=True
print('Least cloudy image:')
image.getInfo()

In [ ]:
# Para visualizar el porcentaje de nubes como lista
img_col.aggregate_array('CLOUD_COVER').getInfo() # 1.65

img_col.aggregate_array('CLOUD_COVER_LAND').getInfo() # 3.8%

#### ***5. Visualización de mejor imagen***


##### Sentinel-2

In [ ]:
# @title Visualización rápida Sentinel-2
# @markdown Primera imagen registrada: `2018-12-17`

#@markdown 1. Seleccionamos las fechas de interés:
Start_date = '2012-12-17'  #@param {type: "date"}
End_date   = '2021-02-11'  #@param {type: "date"}
range_date = [Start_date, End_date]

img_s2 = ee.ImageCollection('COPERNICUS/S2_SR')\
        .filterDate(Start_date, End_date)\
        .filterBounds(ee.Geometry.Point(-76.13, -13.48))\
        .sort('CLOUDY_PIXEL_PERCENTAGE')
# [-74.904844,-12.401923]
# Chincha: -76.13, -13.48
# Lima   : -76.94,-12.06
#img_s2.getInfo().keys() # dict_keys(['type', 'bands', 'id', 'version', 'properties'])
#img_s2.bandNames().getInfo()

bands = ['B4','B3','B2']
# img_s2_rgb_proyecto = img_s2.first().select(bands).clip(limite)
img_s2_rgb = img_s2.first().multiply(0.0001) # .clip(roi_clip)
# Fecha de toma
ver_rangos(img_s2, range_date=range_date)
# ID
print()
print('Mejor imagen:')
print('% nubosidad: ', img_s2.first().getInfo()['properties']['CLOUDY_PIXEL_PERCENTAGE'])
print('Image ID:', img_s2.first().getInfo().get('id'))
print('Fecha:'   , conv_fecha(img_s2.first()))

# Visualizacion: Mapdisplay
dict_img_s2 = {'img_s2': img_s2_rgb.getMapId({'min':0.0, 'max':0.3, 'bands': bands})}
mapdisplay(centroide, dict_img_s2, zoom_start=12)

Total imágenes: 156
Rango Temporal Ingresado : 2012-12-17 - 2021-02-11
Rango Temporal Real      : 2018-12-17 - 2021-02-09

Mejor imagen:
% nubosidad:  0.200305
Image ID: COPERNICUS/S2_SR/20190501T151711_20190501T151806_T18LUL
Fecha: 2019-05-01 15:26:50


##### Landsat 8

In [ ]:
#@title Visualización rápida Landsat8
# @markdown Primera imagen registrada: `2013-03-24`
#@markdown 1. Seleccionamos las fechas de interés:

Start_date = '2011-12-17'  #@param {type: "date"}
End_date   = '2021-02-11'  #@param {type: "date"}
range_date = [Start_date, End_date]

img_col = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR")\
        .filterDate(Start_date, End_date)\
        .filterBounds(ee.Geometry.Point(-76.13, -13.48))\
        .sort('CLOUD_COVER')#\.first()

bands = ['B4','B3','B2']
img_rgb = img_col.first().multiply(0.0001).select(bands)   #.clip(roi_clip)
# --- Visualización de fechas ----------
#ver_rangos(img_rgb, range_date=range_date)

# --- Visualizacion de imagen ----------

# Fecha de toma
ver_rangos(img_col, range_date=range_date)
# ID
print()
print('Mejor imagen:')
print('% nubosidad:', img_col.first().getInfo()['properties']['CLOUD_COVER'])
print('Image ID:',img_col.first().getInfo().get('id'))
print('Fecha:', conv_fecha(img_col.first()))


# Visualizacion: Mapdisplay
dict_img = {'img': img_rgb.getMapId({'min':0.0, 'max':0.3, 'bands': bands})} # ,'roi': roi_clip.getInfo()
mapdisplay(centroide, dict_img, zoom_start=12)

Total imágenes: 176
Rango Temporal Ingresado : 2011-12-17 - 2021-02-11
Rango Temporal Real      : 2013-03-24 - 2021-01-29

Mejor imagen:
% nubosidad: 1.65
Image ID: LANDSAT/LC08/C01/T1_SR/LC08_007069_20160405
Fecha: 2016-04-05 15:10:59


##### Landsat 7

In [ ]:
# @title Visualización rápida Landsat 7
# @markdown Primera Imagen registrada: `1999-08-05` (Falló en: May 31, 2003)
# @markdown 1. Seleccionamos las fechas de interés:
Start_date = '1999-01-01'  #@param {type: "date"}
End_date   = '2021-02-12'  #@param {type: "date"}
range_date = [Start_date, End_date]

img_col = ee.ImageCollection("LANDSAT/LE07/C01/T1_SR")\
        .filterDate(Start_date, End_date)\
        .filterBounds(ee.Geometry.Point(-77.03, -13.48))\
        .sort('CLOUD_COVER')#\.first()

bands = ['B3','B2','B1']
img_rgb = img_col.first().multiply(0.0001).select(bands)   #.clip(roi_clip)
# --- Visualización de fechas ----------
ver_rangos(img_col, range_date=range_date)

# --- Visualizacion de imagen ----------
# ID
print()
print('Mejor imagen:')
print('% nubosidad:', img_col.first().getInfo()['properties']['CLOUD_COVER'])
print('Image ID:',img_col.first().getInfo().get('id'))
print('Fecha:', conv_fecha(img_col.first()))

# Visualizacion: Mapdisplay
dict_img = {'img': img_rgb.getMapId({'min':0.0, 'max':0.3, 'bands': bands})} # ,'roi': roi_clip.getInfo()
mapdisplay(centroide, dict_img, zoom_start=12)

Total imágenes: 419
Rango Temporal Ingresado : 1999-01-01 - 2021-02-12
Rango Temporal Real      : 1999-08-05 - 2021-02-06

Mejor imagen:
% nubosidad: 0
Image ID: LANDSAT/LE07/C01/T1_SR/LE07_007069_20060520
Fecha: 2006-05-20 15:01:25


### Pruebas

1. Selección de mejores imágenes
2. Aplicación de máscara de nubes
3. Aplicación de Índices de Vegetación

#### **Prueba 1 - Selección de mejores imágenes**



> Filtrado en ee.ImageCollection : [Filtering](https://developers.google.com/earth-engine/guides/ic_visualization#filtering)

- Ejemplo de filtrado de porcentaje de nubes a nivel mundial:
```
s2col = ee.ImageCollection('COPERNICUS/S2_SR').filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 0.1))
s2col.size().getInfo() # 570616 
s2col = ee.ImageCollection('COPERNICUS/S2_SR').filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 100))
s2col.size().getInfo() # 8241221 WOW!!!
```

##### ***1. Visualización de mejores imágenes Sentinel-2 por mes según % de nubosidad(2019-2020)***

In [ ]:
@markdown *Visualización de imágenes Sentinel-2 (2019-2020)*
# 1. Fechas: 12 elementos en cada lista 
meses_2019 = create_dateranges('2019','month', 1, 12) 
meses_2020 = create_dateranges('2020','month', 1, 12)

# 2. Filtrado de ImageCollection
punto = [-76.13, -13.48]

imgs_2019 = [ee.ImageCollection('COPERNICUS/S2_SR')\
             .filterDate(meses_2019[i][0], meses_2019[i][1])\
             .filterBounds(ee.Geometry.Point(punto)) for i in range(12)] # range(len(meses_2019))

imgs_2020 = [ee.ImageCollection('COPERNICUS/S2_SR')\
             .filterDate(meses_2020[i][0], meses_2020[i][1])\
             .filterBounds(ee.Geometry.Point(punto)) for i in range(12)] # range(len(meses_2020))

# Seleccionar bandas (opcional): bands = ['B4','B3','B2', 'B8', 'B11']

# 3. Ordenar por nubosidad: .sort('CLOUDY_PIXEL_PERCENTAGE', opt_ascending= True) -> Menor Nubosidad 
imgs_2019_best = [imgs_2019[i].sort('CLOUDY_PIXEL_PERCENTAGE').first() for i in range(12)] #.select(bands)
imgs_2020_best = [imgs_2020[i].sort('CLOUDY_PIXEL_PERCENTAGE').first() for i in range(12)] # ee.Image()

imgs_2019_info = [imgs_2019_best[i].getInfo() for i in range(12)] # list class -> 12 images
imgs_2020_info = [imgs_2020_best[i].getInfo() for i in range(12)] # list class -> 12 images

# ------ Visualización RGB ---------------

# imgs_2019_best_1 = [imgs_2019_best[i].clip(roi_clip).divide(10000) for i in range(12)] #pprint(imgs_2019_best_1)
# imgs_2020_best_1 = [imgs_2020_best[i].clip(roi_clip).divide(10000) for i in range(12)] #pprint(imgs_2020_best_1)

# --------------------------------- PROPIEDADES --------------------------------------------------
fechas19 = [meses_2019[i][0] for i in range(12)]
fechas20 = [meses_2020[i][0] for i in range(12)]

imgs_2019_size = [imgs_2019[i].size().getInfo() for i in range(12)]
imgs_2020_size = [imgs_2020[i].size().getInfo() for i in range(12)]

imgs_2019_clouds = [imgs_2019_info[i]['properties']['CLOUDY_PIXEL_PERCENTAGE'] for i in range(12)]
imgs_2020_clouds = [imgs_2020_info[i]['properties']['CLOUDY_PIXEL_PERCENTAGE'] for i in range(12)]

imgs_2019_id = [imgs_2019_info[i]['id'] for i in range(12)]
imgs_2020_id = [imgs_2020_info[i]['id'] for i in range(12)]

imgs_2019_dates = [conv_fecha(imgs_2019_info[i], getInfo=True) for i in range(12)] 
imgs_2020_dates = [conv_fecha(imgs_2020_info[i], getInfo=True) for i in range(12)]

# Para visualizar por completo cualquier propiedad: print() 
fecha  = fechas19 + fechas20
imgs   = imgs_2019_size + imgs_2020_size
clouds = imgs_2019_clouds + imgs_2020_clouds
id     = imgs_2019_id + imgs_2020_id
dates  = imgs_2019_dates + imgs_2020_dates

# Propiedades de la mejor imagen según porcentaje de nubosidad (DataFrame)
data = {'Fecha': fecha,
        'Imgs': imgs,
        'Clouds Perc (%)': clouds,
        'Time-start': dates} # 'ID best Image': id

# -------- DataFrame Section -----------------
import pandas as pd
df = pd.DataFrame.from_dict(data) 
# print('DataFrame Name: data\n', )
display(df)
# df.head(25)

sum_19 = 0
for i in range(12):
  sum_19 =+ sum_19+ data['Imgs'][i]
print('Imagenes 2019:', sum_19)

sum_20 = 0
for i in range(13,24):
  sum_20 =+ sum_20 + data['Imgs'][i]

print('Imagenes 2020:',sum_20)
print('Total:', sum_19+sum_20)

# df.to_csv('busqueda_sentinel_2.csv')

,Fecha,Imgs,Clouds Perc (%),Time-start
0,2019-01,7,11.442228,2019-01-26 15:26:47
1,2019-02,5,9.898520,2019-02-05 15:26:47
2,2019-03,6,2.984784,2019-03-17 15:26:45
3,2019-04,6,0.355404,2019-04-11 15:26:48
4,2019-05,7,0.200305,2019-05-01 15:26:50
5,2019-06,5,61.382124,2019-06-15 15:26:54
6,2019-07,6,25.321462,2019-07-25 15:26:55
7,2019-08,6,52.501521,2019-08-19 15:26:49
8,2019-09,6,38.420651,2019-09-08 15:26:45
9,2019-10,6,34.763734,2019-10-28 15:26:50


Imagenes 2019: 72
Imagenes 2020: 67
Total: 139


In [ ]:
def add_date_info(df):
  # df['Timestamp'] = pd.to_datetime(df['Time-start'], unit='ms')
  df['Year'] = pd.DatetimeIndex(df['Time-start']).year
  df['Month'] = pd.DatetimeIndex(df['Time-start']).month
  df['Day'] = pd.DatetimeIndex(df['Time-start']).day
  df['DOY'] = pd.DatetimeIndex(df['Time-start']).dayofyear
  return df

add_date_info(df)
df = df.drop(columns=['Time-start'])
df

,Fecha,Imgs,Clouds Perc (%),Year,Month,Day,DOY
0,2019-01,7,11.442228,2019,1,26,26
1,2019-02,5,9.898520,2019,2,5,36
2,2019-03,6,2.984784,2019,3,17,76
3,2019-04,6,0.355404,2019,4,11,101
4,2019-05,7,0.200305,2019,5,1,121
5,2019-06,5,61.382124,2019,6,15,166
6,2019-07,6,25.321462,2019,7,25,206
7,2019-08,6,52.501521,2019,8,19,231
8,2019-09,6,38.420651,2019,9,8,251
9,2019-10,6,34.763734,2019,10,28,301


###### Visualizaciones

In [ ]:
# @markdown Diccionarios generados: `imgs_2019_id`, `imgs_2020_id` 
imgs_2019_id = ['COPERNICUS/S2_SR/20190126T151709_20190126T151946_T18LUL',
                'COPERNICUS/S2_SR/20190205T151709_20190205T152124_T18LUL',
                'COPERNICUS/S2_SR/20190317T151659_20190317T151944_T18LUL',
                'COPERNICUS/S2_SR/20190411T151711_20190411T152328_T18LUL',
                'COPERNICUS/S2_SR/20190501T151711_20190501T151806_T18LUL',
                'COPERNICUS/S2_SR/20190615T151709_20190615T152011_T18LUL',
                'COPERNICUS/S2_SR/20190725T151709_20190725T152436_T18LUL',
                'COPERNICUS/S2_SR/20190819T151711_20190819T151812_T18LUL',
                'COPERNICUS/S2_SR/20190908T151701_20190908T151801_T18LUL',
                'COPERNICUS/S2_SR/20191028T151711_20191028T152253_T18LUL',
                'COPERNICUS/S2_SR/20191102T151709_20191102T151711_T18LUL',
                'COPERNICUS/S2_SR/20191227T151701_20191227T152458_T18LUL']

imgs_2020_id = ['COPERNICUS/S2_SR/20200111T151659_20200111T151911_T18LUL',
                'COPERNICUS/S2_SR/20200205T151701_20200205T151919_T18LUL',
                'COPERNICUS/S2_SR/20200311T151659_20200311T152350_T18LUL',
                'COPERNICUS/S2_SR/20200420T151659_20200420T151727_T18LUL',
                'COPERNICUS/S2_SR/20200510T151659_20200510T152614_T18LUL',
                'COPERNICUS/S2_SR/20200619T151709_20200619T151919_T18LUL',
                'COPERNICUS/S2_SR/20200724T151711_20200724T151814_T18LUL',
                'COPERNICUS/S2_SR/20200813T151711_20200813T151800_T18LUL',
                'COPERNICUS/S2_SR/20200927T151709_20200927T151805_T18LUL',
                'COPERNICUS/S2_SR/20201022T151711_20201022T152346_T18LUL',
                'COPERNICUS/S2_SR/20201111T151711_20201111T152537_T18LUL',
                'COPERNICUS/S2_SR/20201221T151701_20201221T152622_T18LUL']

In [ ]:
# Composición de Bandas: Sentinel-2
# Simple RGB Composites (Sentinel-2): https://github.com/sentinel-hub/custom-scripts/tree/master/sentinel-2/composites

True_Color          = ['B4', 'B3', 'B2'] # Natural Color
Color_Infrared      = ['B8', 'B4', 'B3'] # False Color
Short_Wave_Infrared = ['B12', 'B8A', 'B4']
Agriculture         = ['B11', 'B8', 'B2']
Geology             = ['B12', 'B11', 'B2']
Bathymetric         = ['B4', 'B3', 'B1']

In [ ]:
# @title Visualización de Mejores imágenes Sentinel-2 (según % de nubosidad)
Product_name = imgs_2019_id #@param ['imgs_2019_id', 'imgs_2020_id'] {type:"raw"}
Bands_composite = Short_Wave_Infrared #@param ['True_Color', 'Color_Infrared', 'Short_Wave_Infrared', 'Agriculture', 'Geology', 'Bathymetric'] {type:"raw"}

# Diccionario para visualización 2019
vis = {}
for i in range(12):
  vis[
      conv_fecha(ee.Image(Product_name[i])), 
      ee.Image(Product_name[i]).getInfo()['properties']['CLOUDY_PIXEL_PERCENTAGE']
      ] = \
      ee.Image(Product_name[i]).multiply(0.0001).getMapId({'min':0,'max':0.3, 'bands':Bands_composite})
# vis['area total'] = area_total.geometry().bounds().getInfo()

# Visualización de imágenes
mapdisplay(centroide, vis, zoom_start=12)

##### ***2. Visualización de imágenes Sentinel-2 (2019)***

In [ ]:
# @markdown Generación de diccionario: 2019 (Generar) (3min 7s aprox.)
meses_2019 = create_dateranges('2019','month', 1, 12)

# Filtrar ImageCollection con list comprenhension
imgs_2019 = [
    ee.ImageCollection('COPERNICUS/S2_SR')
    .filterDate(meses_2019[i][0], meses_2019[i][1])
    .filterBounds(ee.Geometry.Point(-76.13, -13.48)) for i in range(12)
    ]

# Listar cantidad de imágenes por mes
imgs_2019_size = [imgs_2019[i].size().getInfo() for i in range(12)]

# Generar diccionario de IDs
icol_monthly_19 = {}
for i in range(12):
  IDs = []
  icol_monthly_19[i+1] = IDs
  for j in range(imgs_2019_size[i]):
    IDs.append(ee.Image(imgs_2019[i].getInfo()['features'][j]['id'])) 
    # Quitar el ee.Image() para obtener solamente IDs

# icol_monthly_19

In [ ]:
# @markdown Diccionario de IDs - Año 2019 (**Sin Depurar**): **`icol_monthly_19`**
icol_monthly_19 = {
    1: [
        'COPERNICUS/S2_SR/20190101T151701_20190101T152509_T18LUL',
        'COPERNICUS/S2_SR/20190106T151659_20190106T152408_T18LUL',
        'COPERNICUS/S2_SR/20190111T151701_20190111T152441_T18LUL',
        'COPERNICUS/S2_SR/20190116T151709_20190116T152029_T18LUL',
        'COPERNICUS/S2_SR/20190121T151701_20190121T152511_T18LUL',
        'COPERNICUS/S2_SR/20190126T151709_20190126T151946_T18LUL',
        'COPERNICUS/S2_SR/20190131T151701_20190131T152414_T18LUL'
    ],
    2: [
        'COPERNICUS/S2_SR/20190205T151709_20190205T152124_T18LUL',
        'COPERNICUS/S2_SR/20190210T151701_20190210T152316_T18LUL',
        'COPERNICUS/S2_SR/20190215T151709_20190215T152351_T18LUL',
        'COPERNICUS/S2_SR/20190220T151701_20190220T152217_T18LUL',
        'COPERNICUS/S2_SR/20190225T151659_20190225T152600_T18LUL'
    ],
    3: [
        'COPERNICUS/S2_SR/20190302T151701_20190302T152249_T18LUL',
        'COPERNICUS/S2_SR/20190307T151659_20190307T152331_T18LUL',
        'COPERNICUS/S2_SR/20190312T151701_20190312T152039_T18LUL',
        'COPERNICUS/S2_SR/20190317T151659_20190317T151944_T18LUL',
        'COPERNICUS/S2_SR/20190322T151701_20190322T152502_T18LUL',
        'COPERNICUS/S2_SR/20190327T151709_20190327T152150_T18LUL'
    ],
    4: [
        'COPERNICUS/S2_SR/20190401T151701_20190401T151805_T18LUL',
        'COPERNICUS/S2_SR/20190406T151709_20190406T151801_T18LUL',
        'COPERNICUS/S2_SR/20190411T151711_20190411T152328_T18LUL',
        'COPERNICUS/S2_SR/20190416T151709_20190416T151709_T18LUL',
        'COPERNICUS/S2_SR/20190421T151711_20190421T151707_T18LUL',
        'COPERNICUS/S2_SR/20190426T151709_20190426T151711_T18LUL'
    ],
    5: [
        'COPERNICUS/S2_SR/20190501T151711_20190501T151806_T18LUL',
        'COPERNICUS/S2_SR/20190506T151709_20190506T152222_T18LUL',
        'COPERNICUS/S2_SR/20190511T151711_20190511T152359_T18LUL',
        'COPERNICUS/S2_SR/20190516T151709_20190516T152114_T18LUL',
        'COPERNICUS/S2_SR/20190521T151711_20190521T152413_T18LUL',
        'COPERNICUS/S2_SR/20190526T151709_20190526T151712_T18LUL',
        'COPERNICUS/S2_SR/20190531T151711_20190531T151804_T18LUL'
    ],
    6: [
        'COPERNICUS/S2_SR/20190605T151709_20190605T151911_T18LUL',
        'COPERNICUS/S2_SR/20190610T151711_20190610T151847_T18LUL',
        'COPERNICUS/S2_SR/20190615T151709_20190615T152011_T18LUL',
        'COPERNICUS/S2_SR/20190620T151711_20190620T151730_T18LUL',
        'COPERNICUS/S2_SR/20190630T151711_20190630T152556_T18LUL'
    ],
    7: [
        'COPERNICUS/S2_SR/20190705T151709_20190705T151712_T18LUL',
        'COPERNICUS/S2_SR/20190710T151711_20190710T151846_T18LUL',
        'COPERNICUS/S2_SR/20190715T151709_20190715T151712_T18LUL',
        'COPERNICUS/S2_SR/20190720T151711_20190720T151709_T18LUL',
        'COPERNICUS/S2_SR/20190725T151709_20190725T152436_T18LUL',
        'COPERNICUS/S2_SR/20190730T151711_20190730T151807_T18LUL'
    ],
    8: [
        'COPERNICUS/S2_SR/20190804T151709_20190804T151711_T18LUL',
        'COPERNICUS/S2_SR/20190809T151711_20190809T151806_T18LUL',
        'COPERNICUS/S2_SR/20190814T151709_20190814T152546_T18LUL',
        'COPERNICUS/S2_SR/20190819T151711_20190819T151812_T18LUL',
        'COPERNICUS/S2_SR/20190824T151709_20190824T152602_T18LUL',
        'COPERNICUS/S2_SR/20190829T151711_20190829T152353_T18LUL'
    ],
    9: [
        'COPERNICUS/S2_SR/20190903T151709_20190903T152220_T18LUL',
        'COPERNICUS/S2_SR/20190908T151701_20190908T151801_T18LUL',
        'COPERNICUS/S2_SR/20190913T151709_20190913T152341_T18LUL',
        'COPERNICUS/S2_SR/20190918T151701_20190918T151704_T18LUL',
        'COPERNICUS/S2_SR/20190923T151709_20190923T151819_T18LUL',
        'COPERNICUS/S2_SR/20190928T151711_20190928T151940_T18LUL'
    ],
    10: [
        'COPERNICUS/S2_SR/20191003T151709_20191003T152153_T18LUL',
        'COPERNICUS/S2_SR/20191008T151711_20191008T152046_T18LUL',
        'COPERNICUS/S2_SR/20191013T151709_20191013T152534_T18LUL',
        'COPERNICUS/S2_SR/20191018T151711_20191018T151707_T18LUL',
        'COPERNICUS/S2_SR/20191023T151709_20191023T152603_T18LUL',
        'COPERNICUS/S2_SR/20191028T151711_20191028T152253_T18LUL'
    ],
    11: [
        'COPERNICUS/S2_SR/20191102T151709_20191102T151711_T18LUL',
        'COPERNICUS/S2_SR/20191107T151711_20191107T152351_T18LUL',
        'COPERNICUS/S2_SR/20191112T151709_20191112T151801_T18LUL',
        'COPERNICUS/S2_SR/20191117T151711_20191117T152440_T18LUL',
        'COPERNICUS/S2_SR/20191122T151659_20191122T151842_T18LUL',
        'COPERNICUS/S2_SR/20191127T151701_20191127T152504_T18LUL'
    ],
    12: [
        'COPERNICUS/S2_SR/20191202T151659_20191202T151911_T18LUL',
        'COPERNICUS/S2_SR/20191207T151701_20191207T152501_T18LUL',
        'COPERNICUS/S2_SR/20191212T151659_20191212T152415_T18LUL',
        'COPERNICUS/S2_SR/20191217T151701_20191217T152458_T18LUL',
        'COPERNICUS/S2_SR/20191222T151659_20191222T151941_T18LUL',
        'COPERNICUS/S2_SR/20191227T151701_20191227T152458_T18LUL'
    ]
}

In [ ]:
# @markdown Diccionario de IDs - Año 2019 (**Depurado**): 
icol_monthly_19 = {
    1: [
        'COPERNICUS/S2_SR/20190101T151701_20190101T152509_T18LUL',
        'COPERNICUS/S2_SR/20190106T151659_20190106T152408_T18LUL',
        'COPERNICUS/S2_SR/20190111T151701_20190111T152441_T18LUL',
        'COPERNICUS/S2_SR/20190116T151709_20190116T152029_T18LUL',
        'COPERNICUS/S2_SR/20190121T151701_20190121T152511_T18LUL',
        'COPERNICUS/S2_SR/20190126T151709_20190126T151946_T18LUL',
        'COPERNICUS/S2_SR/20190131T151701_20190131T152414_T18LUL'
    ],
    2: [
        'COPERNICUS/S2_SR/20190205T151709_20190205T152124_T18LUL',
        'COPERNICUS/S2_SR/20190210T151701_20190210T152316_T18LUL',
        'COPERNICUS/S2_SR/20190215T151709_20190215T152351_T18LUL',
        'COPERNICUS/S2_SR/20190220T151701_20190220T152217_T18LUL',
        'COPERNICUS/S2_SR/20190225T151659_20190225T152600_T18LUL'
    ],
    3: [
        'COPERNICUS/S2_SR/20190302T151701_20190302T152249_T18LUL',
        'COPERNICUS/S2_SR/20190307T151659_20190307T152331_T18LUL',
        'COPERNICUS/S2_SR/20190312T151701_20190312T152039_T18LUL',
        'COPERNICUS/S2_SR/20190317T151659_20190317T151944_T18LUL',
        'COPERNICUS/S2_SR/20190322T151701_20190322T152502_T18LUL',
        'COPERNICUS/S2_SR/20190327T151709_20190327T152150_T18LUL'
    ],
    4: [
        'COPERNICUS/S2_SR/20190401T151701_20190401T151805_T18LUL',
        'COPERNICUS/S2_SR/20190406T151709_20190406T151801_T18LUL',
        'COPERNICUS/S2_SR/20190411T151711_20190411T152328_T18LUL',
        'COPERNICUS/S2_SR/20190416T151709_20190416T151709_T18LUL',
        'COPERNICUS/S2_SR/20190421T151711_20190421T151707_T18LUL',
        'COPERNICUS/S2_SR/20190426T151709_20190426T151711_T18LUL'
    ],
    5: [
        'COPERNICUS/S2_SR/20190501T151711_20190501T151806_T18LUL',
        'COPERNICUS/S2_SR/20190506T151709_20190506T152222_T18LUL',
        'COPERNICUS/S2_SR/20190511T151711_20190511T152359_T18LUL',
        'COPERNICUS/S2_SR/20190516T151709_20190516T152114_T18LUL',
        'COPERNICUS/S2_SR/20190521T151711_20190521T152413_T18LUL',
        'COPERNICUS/S2_SR/20190526T151709_20190526T151712_T18LUL',
        'COPERNICUS/S2_SR/20190531T151711_20190531T151804_T18LUL'
    ],
    6: [
        'COPERNICUS/S2_SR/20190605T151709_20190605T151911_T18LUL',
        'COPERNICUS/S2_SR/20190610T151711_20190610T151847_T18LUL',
        'COPERNICUS/S2_SR/20190615T151709_20190615T152011_T18LUL',
        'COPERNICUS/S2_SR/20190620T151711_20190620T151730_T18LUL',
        'COPERNICUS/S2_SR/20190630T151711_20190630T152556_T18LUL'
    ],
    7: [
        'COPERNICUS/S2_SR/20190705T151709_20190705T151712_T18LUL',
        'COPERNICUS/S2_SR/20190710T151711_20190710T151846_T18LUL',
        'COPERNICUS/S2_SR/20190715T151709_20190715T151712_T18LUL',
        'COPERNICUS/S2_SR/20190720T151711_20190720T151709_T18LUL',
        'COPERNICUS/S2_SR/20190725T151709_20190725T152436_T18LUL',
        'COPERNICUS/S2_SR/20190730T151711_20190730T151807_T18LUL'
    ],
    8: [
        'COPERNICUS/S2_SR/20190804T151709_20190804T151711_T18LUL',
        'COPERNICUS/S2_SR/20190809T151711_20190809T151806_T18LUL',
        'COPERNICUS/S2_SR/20190814T151709_20190814T152546_T18LUL',
        'COPERNICUS/S2_SR/20190819T151711_20190819T151812_T18LUL',
        'COPERNICUS/S2_SR/20190824T151709_20190824T152602_T18LUL',
        'COPERNICUS/S2_SR/20190829T151711_20190829T152353_T18LUL'
    ],
    9: [
        'COPERNICUS/S2_SR/20190903T151709_20190903T152220_T18LUL',
        'COPERNICUS/S2_SR/20190908T151701_20190908T151801_T18LUL',
        'COPERNICUS/S2_SR/20190913T151709_20190913T152341_T18LUL',
        'COPERNICUS/S2_SR/20190918T151701_20190918T151704_T18LUL',
        'COPERNICUS/S2_SR/20190923T151709_20190923T151819_T18LUL',
        'COPERNICUS/S2_SR/20190928T151711_20190928T151940_T18LUL'
    ],
    10: [
        'COPERNICUS/S2_SR/20191003T151709_20191003T152153_T18LUL',
        'COPERNICUS/S2_SR/20191008T151711_20191008T152046_T18LUL',
        'COPERNICUS/S2_SR/20191013T151709_20191013T152534_T18LUL',
        'COPERNICUS/S2_SR/20191018T151711_20191018T151707_T18LUL',
        'COPERNICUS/S2_SR/20191023T151709_20191023T152603_T18LUL',
        'COPERNICUS/S2_SR/20191028T151711_20191028T152253_T18LUL'
    ],
    11: [
        'COPERNICUS/S2_SR/20191102T151709_20191102T151711_T18LUL',
        'COPERNICUS/S2_SR/20191107T151711_20191107T152351_T18LUL',
        'COPERNICUS/S2_SR/20191112T151709_20191112T151801_T18LUL',
        'COPERNICUS/S2_SR/20191117T151711_20191117T152440_T18LUL',
        'COPERNICUS/S2_SR/20191122T151659_20191122T151842_T18LUL',
        'COPERNICUS/S2_SR/20191127T151701_20191127T152504_T18LUL'
    ],
    12: [
        'COPERNICUS/S2_SR/20191202T151659_20191202T151911_T18LUL',
        'COPERNICUS/S2_SR/20191207T151701_20191207T152501_T18LUL',
        'COPERNICUS/S2_SR/20191212T151659_20191212T152415_T18LUL',
        'COPERNICUS/S2_SR/20191217T151701_20191217T152458_T18LUL',
        'COPERNICUS/S2_SR/20191222T151659_20191222T151941_T18LUL',
        'COPERNICUS/S2_SR/20191227T151701_20191227T152458_T18LUL'
    ]
}

In [ ]:
def dividir_y_cortar(image, roi_clip):
  """Retorna una imagen dividida entre 10000 y cortada por un roi_clip."""
  return image.divide(10000).clip(roi_clip)

def get_cloud_perc(image):
  """Retorna el porcentaje de nubosidad de una imagen."""
  return image.getInfo()['properties']['CLOUDY_PIXEL_PERCENTAGE']

In [ ]:
# @markdown Visualización de Imágenes 2019: Por meses

mes = 1 #@param [1,2,3,4,5,6,7,8,9,10,11,12] {type:"raw"}
Natural_Color  = ['B4', 'B3', 'B2']

# Diccionario para visualización
mydict_19 = {}
for i in range(len(icol_monthly_19[mes])):
  mydict_19[
    conv_fecha(ee.Image(icol_monthly_19[mes][i])),  
    get_cloud_perc(ee.Image(icol_monthly_19[mes][i])) 
    ] \
    = ee.Image(icol_monthly_19[mes][i])\
    .divide(10000)\
    .clip(roi_clip)\
    .getMapId({'min':0,'max':0.3,'bands':Natural_Color})

mapdisplay(centroide, mydict_19, zoom_start=12)

##### ***3. Visualización de imágenes Sentinel2 (2020)***

In [ ]:
# @markdown Generación de diccionario: 2020 (Generar) 
meses_2020 = create_dateranges('2020','month', 1, 12)

# Filtro de ImageCollection (list comprenhension)
imgs_2020 = [ee.ImageCollection('COPERNICUS/S2_SR')
             .filterDate(meses_2020[i][0], meses_2020[i][1])\
             .filterBounds(ee.Geometry.Point(-76.13, -13.48)) for i in range(12)]

# Cantidad de imágenes por mes 
imgs_2020_size = [imgs_2020[i].size().getInfo() for i in range(12)]

# Diccionario de Meses con su imágenes correspondientes
icol_monthly_20 = {}
for i in range(12):
  IDs = []
  icol_monthly_20[i+1] = IDs
  for j in range(imgs_2020_size[i]):
    IDs.append(imgs_2020[i].getInfo()['features'][j]['id']) 
    # IDs.append(ee.Image(imgs_2020[i].getInfo()['features'][j]['id'])) 
    
    # Quitar el ee.Image() para obtener solamente IDs

# icol_monthly_20

In [ ]:
# @markdown Diccionario de IDs - Año 2020 (**Sin depurar**)
# @markdown - **`icol_monthly_20`**
icol_monthly_20 = {
    1: [
        'COPERNICUS/S2_SR/20200101T151659_20200101T151930_T18LUL',
        'COPERNICUS/S2_SR/20200106T151701_20200106T152458_T18LUL',
        'COPERNICUS/S2_SR/20200111T151659_20200111T151911_T18LUL',
        'COPERNICUS/S2_SR/20200116T151701_20200116T152038_T18LUL',
        'COPERNICUS/S2_SR/20200121T151659_20200121T152634_T18LUL',
        'COPERNICUS/S2_SR/20200126T151701_20200126T152012_T18LUL'
    ],
    2: [
        'COPERNICUS/S2_SR/20200205T151701_20200205T151919_T18LUL',
        'COPERNICUS/S2_SR/20200210T151659_20200210T152519_T18LUL',
        'COPERNICUS/S2_SR/20200215T151701_20200215T152236_T18LUL',
        'COPERNICUS/S2_SR/20200220T151659_20200220T152051_T18LUL',
        'COPERNICUS/S2_SR/20200225T151701_20200225T152133_T18LUL'
    ],
    3: [
        'COPERNICUS/S2_SR/20200301T151659_20200301T152551_T18LUL',
        'COPERNICUS/S2_SR/20200306T151701_20200306T152018_T18LUL',
        'COPERNICUS/S2_SR/20200311T151659_20200311T152350_T18LUL',
        'COPERNICUS/S2_SR/20200316T151701_20200316T151812_T18LUL',
        'COPERNICUS/S2_SR/20200321T151659_20200321T152227_T18LUL',
        'COPERNICUS/S2_SR/20200326T151701_20200326T152500_T18LUL',
        'COPERNICUS/S2_SR/20200331T151659_20200331T152436_T18LUL'
    ],
    4: [
        'COPERNICUS/S2_SR/20200405T151701_20200405T151744_T18LUL',
        'COPERNICUS/S2_SR/20200410T151659_20200410T152616_T18LUL',
        'COPERNICUS/S2_SR/20200415T151701_20200415T152601_T18LUL',
        'COPERNICUS/S2_SR/20200420T151659_20200420T151727_T18LUL',
        'COPERNICUS/S2_SR/20200425T151711_20200425T151731_T18LUL',
        'COPERNICUS/S2_SR/20200430T151659_20200430T151700_T18LUL'
    ],
    5: [
        'COPERNICUS/S2_SR/20200505T151711_20200505T151755_T18LUL',
        'COPERNICUS/S2_SR/20200510T151659_20200510T152614_T18LUL',
        'COPERNICUS/S2_SR/20200515T151711_20200515T152143_T18LUL',
        'COPERNICUS/S2_SR/20200520T151709_20200520T151704_T18LUL',
        'COPERNICUS/S2_SR/20200525T151711_20200525T151826_T18LUL',
        'COPERNICUS/S2_SR/20200530T151709_20200530T151705_T18LUL'
    ],
    6: [
        'COPERNICUS/S2_SR/20200604T151711_20200604T151847_T18LUL',
        'COPERNICUS/S2_SR/20200609T151709_20200609T151945_T18LUL',
        'COPERNICUS/S2_SR/20200614T151711_20200614T152018_T18LUL',
        'COPERNICUS/S2_SR/20200619T151709_20200619T151919_T18LUL',
        'COPERNICUS/S2_SR/20200624T151711_20200624T151850_T18LUL',
        'COPERNICUS/S2_SR/20200629T151709_20200629T152119_T18LUL'
    ],
    7: [
        'COPERNICUS/S2_SR/20200704T151711_20200704T151850_T18LUL',
        'COPERNICUS/S2_SR/20200709T151709_20200709T152628_T18LUL',
        'COPERNICUS/S2_SR/20200714T151711_20200714T151708_T18LUL',
        'COPERNICUS/S2_SR/20200719T151709_20200719T151705_T18LUL',
        'COPERNICUS/S2_SR/20200724T151711_20200724T151814_T18LUL',
        'COPERNICUS/S2_SR/20200729T151709_20200729T151927_T18LUL'
    ],
    8: [
        'COPERNICUS/S2_SR/20200803T151711_20200803T151800_T18LUL',
        'COPERNICUS/S2_SR/20200808T151709_20200808T152329_T18LUL',
        'COPERNICUS/S2_SR/20200813T151711_20200813T151800_T18LUL',
        'COPERNICUS/S2_SR/20200818T151709_20200818T152416_T18LUL',
        'COPERNICUS/S2_SR/20200823T151711_20200823T152545_T18LUL',
        'COPERNICUS/S2_SR/20200828T151709_20200828T152025_T18LUL'
    ],
    9: [
        'COPERNICUS/S2_SR/20200902T151711_20200902T151709_T18LUL',
        'COPERNICUS/S2_SR/20200902T151711_20200902T152650_T18LUL',
        'COPERNICUS/S2_SR/20200907T151709_20200907T152155_T18LUL',
        'COPERNICUS/S2_SR/20200912T151711_20200912T152508_T18LUL',
        'COPERNICUS/S2_SR/20200917T151709_20200917T151745_T18LUL',
        'COPERNICUS/S2_SR/20200922T151711_20200922T152418_T18LUL',
        'COPERNICUS/S2_SR/20200927T151709_20200927T151805_T18LUL'
    ],
    10: [
        'COPERNICUS/S2_SR/20201002T151711_20201002T151847_T18LUL',
        'COPERNICUS/S2_SR/20201007T151709_20201007T152040_T18LUL',
        'COPERNICUS/S2_SR/20201012T151711_20201012T152202_T18LUL',
        'COPERNICUS/S2_SR/20201017T151709_20201017T152445_T18LUL',
        'COPERNICUS/S2_SR/20201022T151711_20201022T152346_T18LUL',
        'COPERNICUS/S2_SR/20201027T151709_20201027T152550_T18LUL'
    ],
    11: [
        'COPERNICUS/S2_SR/20201101T151711_20201101T152447_T18LUL',
        'COPERNICUS/S2_SR/20201106T151709_20201106T151706_T18LUL',
        'COPERNICUS/S2_SR/20201111T151711_20201111T152537_T18LUL',
        'COPERNICUS/S2_SR/20201121T151711_20201121T152604_T18LUL',
        'COPERNICUS/S2_SR/20201126T151709_20201126T151704_T18LUL'
    ],
    12: [
        'COPERNICUS/S2_SR/20201201T151701_20201201T152622_T18LUL',
        'COPERNICUS/S2_SR/20201206T151659_20201206T151702_T18LUL',
        'COPERNICUS/S2_SR/20201211T151701_20201211T152619_T18LUL',
        'COPERNICUS/S2_SR/20201216T151659_20201216T152557_T18LUL',
        'COPERNICUS/S2_SR/20201221T151701_20201221T152622_T18LUL',
        'COPERNICUS/S2_SR/20201226T151659_20201226T151701_T18LUL',
        'COPERNICUS/S2_SR/20201231T151701_20201231T152623_T18LUL'
    ]
}

In [ ]:
# @markdown Diccionario de IDs - Año 2020 (**Depurado**)  
# @markdown - **`icol_monthly_20`**
# @markdown - Mes 6 eliminado (imágenes llena de nubes)
icol_monthly_20 = {
    1: [
        # 'COPERNICUS/S2_SR/20200101T151659_20200101T151930_T18LUL',
        'COPERNICUS/S2_SR/20200106T151701_20200106T152458_T18LUL',
        'COPERNICUS/S2_SR/20200111T151659_20200111T151911_T18LUL',
        'COPERNICUS/S2_SR/20200116T151701_20200116T152038_T18LUL',
        # 'COPERNICUS/S2_SR/20200121T151659_20200121T152634_T18LUL',
        'COPERNICUS/S2_SR/20200126T151701_20200126T152012_T18LUL'
    ],
    2: [
        'COPERNICUS/S2_SR/20200205T151701_20200205T151919_T18LUL',
        'COPERNICUS/S2_SR/20200210T151659_20200210T152519_T18LUL',
        'COPERNICUS/S2_SR/20200215T151701_20200215T152236_T18LUL',
        # 'COPERNICUS/S2_SR/20200220T151659_20200220T152051_T18LUL',
        'COPERNICUS/S2_SR/20200225T151701_20200225T152133_T18LUL'
    ],
    3: [
        'COPERNICUS/S2_SR/20200301T151659_20200301T152551_T18LUL',
        'COPERNICUS/S2_SR/20200306T151701_20200306T152018_T18LUL',
        'COPERNICUS/S2_SR/20200311T151659_20200311T152350_T18LUL',
        # 'COPERNICUS/S2_SR/20200316T151701_20200316T151812_T18LUL',
        'COPERNICUS/S2_SR/20200321T151659_20200321T152227_T18LUL',
        # 'COPERNICUS/S2_SR/20200326T151701_20200326T152500_T18LUL',
        # 'COPERNICUS/S2_SR/20200331T151659_20200331T152436_T18LUL'
    ],
    4: [
        # 'COPERNICUS/S2_SR/20200405T151701_20200405T151744_T18LUL',
        # 'COPERNICUS/S2_SR/20200410T151659_20200410T152616_T18LUL',
        # 'COPERNICUS/S2_SR/20200415T151701_20200415T152601_T18LUL',
        'COPERNICUS/S2_SR/20200420T151659_20200420T151727_T18LUL',
        'COPERNICUS/S2_SR/20200425T151711_20200425T151731_T18LUL',
        'COPERNICUS/S2_SR/20200430T151659_20200430T151700_T18LUL'
    ],
    5: [
        # 'COPERNICUS/S2_SR/20200505T151711_20200505T151755_T18LUL',
        'COPERNICUS/S2_SR/20200510T151659_20200510T152614_T18LUL',
        'COPERNICUS/S2_SR/20200515T151711_20200515T152143_T18LUL',
        # 'COPERNICUS/S2_SR/20200520T151709_20200520T151704_T18LUL',
        # 'COPERNICUS/S2_SR/20200525T151711_20200525T151826_T18LUL',
        # 'COPERNICUS/S2_SR/20200530T151709_20200530T151705_T18LUL'
    ],
    6: [
        # 'COPERNICUS/S2_SR/20200604T151711_20200604T151847_T18LUL',
        # 'COPERNICUS/S2_SR/20200609T151709_20200609T151945_T18LUL',
        # 'COPERNICUS/S2_SR/20200614T151711_20200614T152018_T18LUL',
        # 'COPERNICUS/S2_SR/20200619T151709_20200619T151919_T18LUL',
        # 'COPERNICUS/S2_SR/20200624T151711_20200624T151850_T18LUL',
        # 'COPERNICUS/S2_SR/20200629T151709_20200629T152119_T18LUL'
    ],
    7: [
        # 'COPERNICUS/S2_SR/20200704T151711_20200704T151850_T18LUL',
        # 'COPERNICUS/S2_SR/20200709T151709_20200709T152628_T18LUL',
        # 'COPERNICUS/S2_SR/20200714T151711_20200714T151708_T18LUL',
        # 'COPERNICUS/S2_SR/20200719T151709_20200719T151705_T18LUL',
        'COPERNICUS/S2_SR/20200724T151711_20200724T151814_T18LUL',
        # 'COPERNICUS/S2_SR/20200729T151709_20200729T151927_T18LUL'
    ],
    8: [
        'COPERNICUS/S2_SR/20200803T151711_20200803T151800_T18LUL',
        # 'COPERNICUS/S2_SR/20200808T151709_20200808T152329_T18LUL',
        'COPERNICUS/S2_SR/20200813T151711_20200813T151800_T18LUL',
        # 'COPERNICUS/S2_SR/20200818T151709_20200818T152416_T18LUL',
        # 'COPERNICUS/S2_SR/20200823T151711_20200823T152545_T18LUL',
        # 'COPERNICUS/S2_SR/20200828T151709_20200828T152025_T18LUL'
    ],
    9: [
        # 'COPERNICUS/S2_SR/20200902T151711_20200902T151709_T18LUL',
        # 'COPERNICUS/S2_SR/20200902T151711_20200902T152650_T18LUL',
        # 'COPERNICUS/S2_SR/20200907T151709_20200907T152155_T18LUL',
        # 'COPERNICUS/S2_SR/20200912T151711_20200912T152508_T18LUL',
        # 'COPERNICUS/S2_SR/20200917T151709_20200917T151745_T18LUL',
        # 'COPERNICUS/S2_SR/20200922T151711_20200922T152418_T18LUL',
        'COPERNICUS/S2_SR/20200927T151709_20200927T151805_T18LUL'
    ],
    10: [
        # 'COPERNICUS/S2_SR/20201002T151711_20201002T151847_T18LUL',
        'COPERNICUS/S2_SR/20201007T151709_20201007T152040_T18LUL',
        'COPERNICUS/S2_SR/20201012T151711_20201012T152202_T18LUL',
        # 'COPERNICUS/S2_SR/20201017T151709_20201017T152445_T18LUL',
        'COPERNICUS/S2_SR/20201022T151711_20201022T152346_T18LUL',
        'COPERNICUS/S2_SR/20201027T151709_20201027T152550_T18LUL'
    ],
    11: [
        # 'COPERNICUS/S2_SR/20201101T151711_20201101T152447_T18LUL',
        # 'COPERNICUS/S2_SR/20201106T151709_20201106T151706_T18LUL',
        # 'COPERNICUS/S2_SR/20201111T151711_20201111T152537_T18LUL',
        'COPERNICUS/S2_SR/20201121T151711_20201121T152604_T18LUL',
        'COPERNICUS/S2_SR/20201126T151709_20201126T151704_T18LUL'
    ],
    12: [
        'COPERNICUS/S2_SR/20201201T151701_20201201T152622_T18LUL',
        'COPERNICUS/S2_SR/20201206T151659_20201206T151702_T18LUL',
        'COPERNICUS/S2_SR/20201211T151701_20201211T152619_T18LUL',
        'COPERNICUS/S2_SR/20201216T151659_20201216T152557_T18LUL',
        'COPERNICUS/S2_SR/20201221T151701_20201221T152622_T18LUL',
        'COPERNICUS/S2_SR/20201226T151659_20201226T151701_T18LUL',
        # 'COPERNICUS/S2_SR/20201231T151701_20201231T152623_T18LUL'
    ]
}

In [ ]:
#@markdown Visualización de Imágenes 2020: Por meses
mes = 1 #@param [1,2,3,4,5,6,7,8,9,10,11,12] {type:"raw"}
Natural_Color  = ['B4', 'B3', 'B2']

# Diccionario para visualización
mydict_20 = {}

for i in range(len(icol_monthly_20[mes])):
  mydict_20[
    conv_fecha(ee.Image(icol_monthly_20[mes][i])),  
    get_cloud_perc(ee.Image(icol_monthly_20[mes][i])) 
    ] \
    = ee.Image(icol_monthly_20[mes][i])\
    .divide(10000)\
    .clip(roi_clip)\
    .getMapId({'min':0,'max':0.3,'bands':Natural_Color})

mapdisplay(centroide, mydict_20, zoom_start=12)

#### **Prueba 2 - Aplicación de máscara de nubes**

- [Obscured by Clouds: A Comparative Analysis of Cloud Removal Methods](https://medium.com/project-canopy/obscured-by-clouds-a-comparative-analysis-of-cloud-removal-methods-b9b1c30c4e1f)

##### Prueba 1: [Usar banda QA](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR) (Método convencional) 

<center>
<img src="https://user-images.githubusercontent.com/16768318/72954392-1b403880-3d90-11ea-9956-15e607f00fcb.png" width = 50%/>
</center>

In [ ]:
# @markdown Función: `maskS2clouds`

def maskS2clouds(image):
  qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloudBitMask = 1 << 10
  cirrusBitMask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
  mask = qa.bitwiseAnd(cloudBitMask).eq(0)\
    .And(qa.bitwiseAnd(cirrusBitMask).eq(0))

  return image.updateMask(mask).divide(10000)

In [ ]:
prueba_nubes = ee.Image('COPERNICUS/S2_SR/20200106T151701_20200106T152458_T18LUL').clip(roi_clip)
resultado_1 = maskS2clouds(prueba_nubes)

mapdisplay(centroide, {'s2_prueba':resultado_1.getMapId({'min':0.0, 'max':0.3, 'bands':['B4','B3','B2']})}, zoom_start=12)

##### Prueba 2: Sentinel-2 Cloud Probability

> **Máscara de nubes con Sentinel-2: Cloud Probability**

Referencias
- [EE: Sentinel-2: Cloud Probability](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_CLOUD_PROBABILITY)
- Turorial para Python: [EE: Sentinel-2 Cloud Masking with s2cloudless](https://developers.google.com/earth-engine/tutorials/community/sentinel-2-s2cloudless)
- Aplicación de [navegador de imágenes S2 SR](https://showcase.earthengine.app/view/s2-sr-browser-s2cloudless-nb) útil para identificar rápidamente imágenes y determinar los criterios de filtro de colección de imágenes.

Blogs:
- [Medium: Cloud Masks at Your Service](https://medium.com/sentinel-hub/cloud-masks-at-your-service-6e5b2cb2ce8a)
- [Medium: On cloud detection with multi-temporal data](https://medium.com/sentinel-hub/on-cloud-detection-with-multi-temporal-data-f64f9b8d59e5)
- [Sentinel-hub: Cloudless](https://docs.sentinel-hub.com/api/latest/user-guides/cloud-masks/#cloud-masks-and-cloud-probabilities)
- [Land Cover Classification with eo-learn: Part 1 - Mastering Satellite Data in an Open-Source Python Environment](https://medium.com/sentinel-hub/land-cover-classification-with-eo-learn-part-1-2471e8098195)
- [Medium: How to Make the Perfect Time-Lapse of the Earth](https://medium.com/sentinel-hub/how-to-make-the-perfect-time-lapse-of-the-earth-351f214527f6)

#### **Prueba 3 - Aplicación de Índices de Vegetación**

In [ ]:
# @markdown Diccionarios generados: `imgs_2019_id`, `imgs_2020_id`
imgs_2019_id = [
    'COPERNICUS/S2_SR/20190126T151709_20190126T151946_T18LUL',
    'COPERNICUS/S2_SR/20190205T151709_20190205T152124_T18LUL',
    'COPERNICUS/S2_SR/20190317T151659_20190317T151944_T18LUL',
    'COPERNICUS/S2_SR/20190411T151711_20190411T152328_T18LUL',
    'COPERNICUS/S2_SR/20190501T151711_20190501T151806_T18LUL',
    'COPERNICUS/S2_SR/20190615T151709_20190615T152011_T18LUL',
    'COPERNICUS/S2_SR/20190725T151709_20190725T152436_T18LUL',
    'COPERNICUS/S2_SR/20190819T151711_20190819T151812_T18LUL',
    'COPERNICUS/S2_SR/20190908T151701_20190908T151801_T18LUL',
    'COPERNICUS/S2_SR/20191028T151711_20191028T152253_T18LUL',
    'COPERNICUS/S2_SR/20191102T151709_20191102T151711_T18LUL',
    'COPERNICUS/S2_SR/20191227T151701_20191227T152458_T18LUL'
]

imgs_2020_id = [
    'COPERNICUS/S2_SR/20200111T151659_20200111T151911_T18LUL',
    'COPERNICUS/S2_SR/20200205T151701_20200205T151919_T18LUL',
    'COPERNICUS/S2_SR/20200311T151659_20200311T152350_T18LUL',
    'COPERNICUS/S2_SR/20200420T151659_20200420T151727_T18LUL',
    'COPERNICUS/S2_SR/20200510T151659_20200510T152614_T18LUL',
    'COPERNICUS/S2_SR/20200619T151709_20200619T151919_T18LUL',
    'COPERNICUS/S2_SR/20200724T151711_20200724T151814_T18LUL',
    'COPERNICUS/S2_SR/20200813T151711_20200813T151800_T18LUL',
    'COPERNICUS/S2_SR/20200927T151709_20200927T151805_T18LUL',
    'COPERNICUS/S2_SR/20201022T151711_20201022T152346_T18LUL',
    'COPERNICUS/S2_SR/20201111T151711_20201111T152537_T18LUL',
    'COPERNICUS/S2_SR/20201221T151701_20201221T152622_T18LUL'
]

In [ ]:
# @markdown **Paletas de colores (html)**
#------------- Paletas de NDVI ---------------------------------------
ndviPalette01 = ['blue', 'white', 'green']

# https://developers.google.com/earth-engine/datasets/catalog/NASA_GIMMS_3GV0?hl=en
ndviPalette02 = ['000000', 'f5f5f5', '119701']

# https://developers.google.com/earth-engine/datasets/catalog/MODIS_006_MOD13A1?hl=en#description
ndviPalette03 = [
    'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
    '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
    '012E01', '011D01', '011301'
]

# Paleta NDVI (Yuri SIG) 20 Colores
# @markdown - `yuri_ndvi_pal`
yuri_ndvi_pal = [
    'ffffff', 'fffcff', 'fff9ff', 'fff7ff', 'fff4ff', 'fff2ff', 'ffefff',
    'ffecff', 'ffeaff', 'ffe7ff', 'ffe5ff', 'ffe2ff', 'ffe0ff', 'ffddff',
    'ffdaff', 'ffd8ff', 'ffd5ff', 'ffd3ff', 'ffd0ff', 'ffceff', 'ffcbff',
    'ffc8ff', 'ffc6ff', 'ffc3ff', 'ffc1ff', 'ffbeff', 'ffbcff', 'ffb9ff',
    'ffb6ff', 'ffb4ff', 'ffb1ff', 'ffafff', 'ffacff', 'ffaaff', 'ffa7ff',
    'ffa4ff', 'ffa2ff', 'ff9fff', 'ff9dff', 'ff9aff', 'ff97ff', 'ff95ff',
    'ff92ff', 'ff90ff', 'ff8dff', 'ff8bff', 'ff88ff', 'ff85ff', 'ff83ff',
    'ff80ff', 'ff7eff', 'ff7bff', 'ff79ff', 'ff76ff', 'ff73ff', 'ff71ff',
    'ff6eff', 'ff6cff', 'ff69ff', 'ff67ff', 'ff64ff', 'ff61ff', 'ff5fff',
    'ff5cff', 'ff5aff', 'ff57ff', 'ff55ff', 'ff52ff', 'ff4fff', 'ff4dff',
    'ff4aff', 'ff48ff', 'ff45ff', 'ff42ff', 'ff40ff', 'ff3dff', 'ff3bff',
    'ff38ff', 'ff36ff', 'ff33ff', 'ff30ff', 'ff2eff', 'ff2bff', 'ff29ff',
    'ff26ff', 'ff24ff', 'ff21ff', 'ff1eff', 'ff1cff', 'ff19ff', 'ff17ff',
    'ff14ff', 'ff12ff', 'ff0fff', 'ff0cff', 'ff0aff', 'ff07ff', 'ff05ff',
    'ff02ff', 'ff00ff', 'ff00ff', 'ff0af4', 'ff15e9', 'ff1fdf', 'ff2ad4',
    'ff35c9', 'ff3fbf', 'ff4ab4', 'ff55aa', 'ff5f9f', 'ff6a94', 'ff748a',
    'ff7f7f', 'ff8a74', 'ff946a', 'ff9f5f', 'ffaa55', 'ffb44a', 'ffbf3f',
    'ffc935', 'ffd42a', 'ffdf1f', 'ffe915', 'fff40a', 'ffff00', 'ffff00',
    'fffb00', 'fff700', 'fff300', 'fff000', 'ffec00', 'ffe800', 'ffe400',
    'ffe100', 'ffdd00', 'ffd900', 'ffd500', 'ffd200', 'ffce00', 'ffca00',
    'ffc600', 'ffc300', 'ffbf00', 'ffbb00', 'ffb700', 'ffb400', 'ffb000',
    'ffac00', 'ffa800', 'ffa500', 'ffa500', 'f7a400', 'f0a300', 'e8a200',
    'e1a200', 'd9a100', 'd2a000', 'ca9f00', 'c39f00', 'bb9e00', 'b49d00',
    'ac9c00', 'a59c00', '9d9b00', '969a00', '8e9900', '879900', '7f9800',
    '789700', '709700', '699600', '619500', '5a9400', '529400', '4b9300',
    '439200', '349100', '2d9000', '258f00', '1e8e00', '168e00', '0f8d00',
    '078c00', '008c00', '008c00', '008700', '008300', '007f00', '007a00',
    '007600', '007200', '006e00', '006900', '006500', '006100', '005c00',
    '005800', '005400', '005000', '004c00'
]

#------------- Paletas de EVI ---------------------------------------
evi_palette = [
    'ffffff', 'fcd163', '99b718', '66a000', '3e8601', '207401', '056201',
    '004c00', '011301'
]

ndvi_palette = [
    "#051852", "#FFFFFF", "#C7B59B", "#A8B255", "#A3C020", "#76AD00",
    "#429001", "#006400", "#003B00", "#000000"
]

In [ ]:
# @markdown Función NDVI 

def ndvi_icol(image):
  ndvi = image.expression(
      '(nir - red) / (nir + red)',
      {'red' : image.select('B4').divide(10000),
       'nir' : image.select('B8').divide(10000)}).rename('NDVI')
  return ndvi

# def ndvi_icol(image):
#   ndvi = image.normalizedDifference(['B8','B4']).rename('ndvi')
#   return image.addBands([ndvi])

In [ ]:
# @markdown Función EVI

# def evi_s2(image):
#   nir  = image.select(['B8']).divide(10000)
#   red  = image.select(['B4']).divide(10000)
#   blue = image.select(['B2']).divide(10000)
#   return nir.subtract(red).divide(nir.add(red.multiply(6)).subtract(blue.multiply(7.5)).add(1)).multiply(2.5).rename('EVI')

def evi_icol(img):
  """
  https://developers.google.com/earth-engine/guides/image_math#expressions
  """
  evi = img.expression('2.5 * (nir - red) / (nir + 6 * red - 7.5 * blue + 1)',
                       {'red' : img.select('B4').divide(10000),
                        'nir' : img.select('B8').divide(10000),
                        'blue': img.select('B2').divide(10000)}).rename('EVI')

  return evi

# https://colab.research.google.com/drive/1GCAoapxKzGbT2HcKfKOIf_WDObumoOqJ#scrollTo=5HVOHGM17_8L

##### ***Índices de Vegetación - Descripción***

Información de Índices de Vegetación

- [The MODIS vegetation index (VI) products](https://lpdaac.usgs.gov/documents/103/MOD13_User_Guide_V6.pdf)

Vegetation Indices (VI) are robust, empirical measures of vegetation activity at the land surface. They are designed to enhance the vegetation reflected signal from measured spectral responses by combining two (or more) wavebands, often in the red(0.6 - 0.7 μm) and NIR wavelengths (0.7 - 1.1 μm) regions.

The second VI product is the Enhanced Vegetation Index (EVI), with improved sensitivity over high biomass regions and  improved vegetation monitoring capability  through a decoupling of the canopy background signal and a reduction in atmosphere influences.

- [Measuring Vegetation (NDVI & EVI)](https://earthobservatory.nasa.gov/features/MeasuringVegetation/measuring_vegetation_3.php)
- [Datasets tagged evi in Earth Engine:](https://developers.google.com/earth-engine/datasets/tags/evi)
  - [MYD13A1.006 Aqua Vegetation Indices 16-Day Global 500m](https://developers.google.com/earth-engine/datasets/catalog/MODIS_006_MYD13A1)
  - [MYD13A2.006 Aqua Vegetation Indices 16-Day Global 1km](https://developers.google.com/earth-engine/datasets/catalog/MODIS_006_MYD13A2)
  - [Oxford MAP EVI: Malaria Atlas Project Gap-Filled Enhanced Vegetation Index](https://developers.google.com/earth-engine/datasets/catalog/Oxford_MAP_EVI_5km_Monthly)

###### **[Vegetation Index - Wikipedia](https://en.wikipedia.org/wiki/Vegetation_Index)**

- A Vegetation Index (VI) is a spectral transformation of two or more bands designed to enhance the contribution of vegetation properties and allow reliable spatial and temporal inter-comparisons of terrestrial photosynthetic activity and canopy structural variations.
- Many of the indices make use of the inverse relationship between red and near-infrared reflectance associated with healthy green vegetation. 
- Since the 1960s scientists have used satellite remote sensing to monitor fluctuation in vegetation at the Earth's surface. Measurements of vegetation attributes include:
  - leaf area index (LAI),
  - percent green cover,
  - chlorophyll content,
  - green biomass and
  - absorbed photosynthetically active radiation (APAR).
- Vegetation Indices (VIs) have been historically classified based on a range of attributes, including:
  - the number of spectral bands (2 or greather than 2),
  - the method of calculations (ratio or orthogonal),
  - depending on the required objective
  - or by their historical development (classified as first generation VIs or second generation VIs)
- For the sake of comparison of the effectiveness of different VIs, Lyon, Yuan et al. (1998) classified 7 VIs based on their computation methods (subtraction, division or rational transform). 

Vegetation indices have been used to:
- examine climate trends
- estimate water content of soils remotely,
- monitor drought
- schedule crop irrigation, crop management,
- monitor evaporation and plant transpiration
- assess changes in biodiversity,
- [classify vegetation](https://www.tandfonline.com/doi/abs/10.1080/01431169008955174)

List of vegetation indices (Multispectral):
- simple ratio
- normalised difference vegetation index (NDVI)
- Kauth-Thomas Tasseled Cap Transformation
- Infrared index
- Perpendicular vegetation index
- greenness above bare soil
- leaf water content index (LWCI)
- MidIR Index
- Soil-adjusted vegetation index (SAVI)
- Modified SAVI
- Atmospherically resistant vegetation index
- Soil and Atmospherically resistant vegetation index
- Enhanced vegetation index (EVI)
- New Vegetation index
- Aerosol free vegetation index
- Triangular vegetation index
- reduced simple ratio
- visible atmospherically resistant index
- normalised difference built-up index
- weighted difference vegetation index (WDVI)
- fraction of absorbed photosynthetically active radiation (FAPAR)
- Normalised difference greenness index (NDGI)

List of vegetation indices (Hyperspectral):
- discrete-band normalised difference vegetation index
- yellowness index
- photochemical reflectance index
- descrete-band normalised difference water index
- red edge position determination
- Crop chlorophyll content prediction
- moment distance index (MDI)

###### **[Overview of the radiometric and biophysical performance of the MODIS vegetation indices](https://www.sciencedirect.com/science/article/abs/pii/S0034425702000962)**

> 

- Two MODIS vegetation indices (VI), the normalized difference vegetation
index (NDVI) and enhanced vegetation index (EVI), are produced at 1-km and 500-m resolutions and 16-day compositing periods.
- Our results
showed a good correspondence between airborne-measured, top-of-canopy reflectances and VI values with those from the MODIS sensor at
four intensively measured test sites representing semi-arid grass/shrub, savanna, and tropical forest biomes.
- Multitemporal profiles of the MODIS VIs over numerous biome
types in North and South America well represented their seasonal phenologies.
- This
requires an understanding of the global distribution of
vegetation types as well as their biophysical and structural
properties and spatial/temporal variations. 
- Vegetation indices (VIs) are spectral transformations of two or more bands
designed to enhance the contribution of vegetation properties and allow reliable spatial and temporal inter-comparisons of terrestrial photosynthetic activity and canopy
structural variations. 
- As a simple transformation of spectral
bands, they are computed directly without any bias or
assumptions regarding land cover class, soil type, or climatic conditions. They allow us to monitor seasonal, interannual, and long-term variations of vegetation structural,
phenological, and biophysical parameters.


###### Otros links:
- [Maximum Likelihood Classification in ArcMap](https://www.coursera.org/lecture/gis-applications/classifying-imagery-and-derived-products-wztEB) 
- [Enhancing green vegetation using mathematical equations and transformations](http://web.pdx.edu/~nauna/resources/8-2012_lecture1-vegetationindicies.pdf)
- https://duckduckgo.com/?q=vegetation+indices+lecture+&ia=web

In [ ]:
meses_2020 = create_dateranges('2020','month', 1, 13)
imgs_2020 = [ee.ImageCollection('COPERNICUS/S2_SR')
             .filterDate(meses_2020[i][0], meses_2020[i][1])\
             .filterBounds(ee.Geometry.Point(-76.13, -13.48)) for i in range(len(meses_2020))]
imgs_2020_size = [imgs_2020[i].size().getInfo() for i in range(12)]

icol_monthly_20 = {}
for i in range(12):
  IDs = []
  icol_monthly_20[i] = IDs
  for j in range(imgs_2020_size[i]):
    IDs.append(ee.Image(imgs_2020[i].getInfo()['features'][j]['id'])) # Quitar el ee.Image() para obtener solamente IDs
# icol_monthly_20

In [ ]:
conv_fecha(icol_monthly_20[0][0])

'2020-01-01 15:26:42'

In [ ]:
#@markdown Visualización de Imágenes 2020: Por meses
mes = 1 #@param [1,2,3,4,5,6,7,8,9,10,11,12] {type:"raw"}
Natural_Color  = ['B4', 'B3', 'B2']

# Diccionario para visualización
mydict_20 = {}
for i in range(len(icol_monthly_20[mes-1])):
  mydict_20[
            i,
            icol_monthly_20[mes][i-1]\
            .getInfo()['properties']['CLOUDY_PIXEL_PERCENTAGE']] = icol_monthly_20[mes][i-1]\
            .clip(roi_clip).divide(10000)\
            .getMapId({'min':0,'max':0.3,'bands':Natural_Color})

print('Cantidad de imágenes del mes {}: {}'.format(mes,len(icol_monthly_20[mes-1])))
# Visualización de mejor imagen según porcentaje de nubosidad
mapdisplay(centroide, mydict_20, zoom_start=12)

##### ***Índices de Vegetación - Aplicación de NDVI***

In [ ]:
# @title Visualización NDVI
Product_name = imgs_2019_id #@param ['imgs_2019_id', 'imgs_2020_id'] {type:"raw"}

# Aplicar el NDVI a todo el año (12 imágenes)
ndvi_prueba = [ndvi_icol(ee.Image(Product_name[i])) for i in range(12)]

# Generar capas para visualización
vi_dict = {}
for i in range(12):
  vi_dict[conv_fecha(ee.Image(Product_name[i]))] = \
  ndvi_prueba[i]\
  .clip(roi_clip)\
  .getMapId({'min':0, 'max':0.5, 'palette': yuri_ndvi_pal})

# En caso de ver el último mes
# vi_dict[conv_fecha(ee.Image(Product_name[11]))] = ndvi_prueba[11].clip(roi_clip).getMapId({'min':0, 'max':0.5, 'palette': yuri_ndvi_pal}) # yuri_ndvi_pal, palette1
# vi_dict[11] = ee.Image(Product_name[11]).multiply(0.0001).clip(roi_clip).getMapId({'min':0, 'max':0.3, 'bands': ['B4','B3','B2']})

mapdisplay(centroide, vi_dict, zoom_start=12)

##### ***Índices de Vegetación - Aplicación de EVI***

In [ ]:
# @title Visualización EVI
Product_name = imgs_2019_id #@param ['imgs_2019_id', 'imgs_2020_id'] {type:"raw"}

# Aplicar el EVI a todo el año (12 imágenes)
evi_prueba = [evi_icol(ee.Image(Product_name[i])) for i in range(12)]

# Generar capas para visualización
vi_dict = {}
for i in range(12):
  vi_dict[conv_fecha(ee.Image(Product_name[i]))]\
  = evi_prueba[i]\
  .clip(roi_clip)\
  .getMapId({'min':0, 'max':0.5, 'palette': yuri_ndvi_pal})

# En caso de ver el último mes
# vi_dict[conv_fecha(ee.Image(Product_name[11]))] = evi_prueba[11].clip(roi_clip).getMapId({'min':0, 'max':0.5, 'palette': yuri_ndvi_pal}) # yuri_ndvi_pal, evi_palette
# vi_dict[11] = ee.Image(Product_name[11]).multiply(0.0001).clip(roi_clip).getMapId({'min':0, 'max':0.3, 'bands': ['B4','B3','B2']})

mapdisplay(centroide, vi_dict, zoom_start=12)

##### ***Índices de Vegetación - Comparación entre NDVI y EVI***

In [ ]:
# @title Comparación NDVI - EVI
Product_name = imgs_2019_id #@param ['imgs_2019_id', 'imgs_2020_id'] {type:"raw"}
mes = 5 #@param [1,2,3,4,5,6,7,8,9,10,11,12] {type:"raw"}
min = "0" #@param [] {allow-input: true}
max = "0.5" #@param [] {allow-input: true}

# ndvi_prueba = [ndvi_icol(ee.Image(Product_name[i])) for i in range(12)]
# evi_prueba  = [ evi_icol(ee.Image(Product_name[i])) for i in range(12)]

dict_prueba = {}

dict_prueba['mes_{a}_rgb' .format(a=mes)] =  ee.Image(Product_name[mes-1]).multiply(0.0001).clip(roi_clip).getMapId({'min':0.0, 'max':0.3, 'bands': ['B4','B3','B2']}) 
dict_prueba['mes_{a}_ndvi'.format(a=mes)] = ndvi_prueba[mes-1].clip(roi_clip).getMapId({'min':min, 'max':max, 'palette': yuri_ndvi_pal}) # yuri_ndvi_pal, palette1
dict_prueba['mes_{a}_evi' .format(a=mes)] =  evi_prueba[mes-1].clip(roi_clip).getMapId({'min':min, 'max':max, 'palette': yuri_ndvi_pal}) # yuri_ndvi_pal, evi_palette

mapdisplay(centroide, dict_prueba, zoom_start=12)

### Visualizaciones mediante gráficos


- [Array and List Charts](https://developers.google.com/earth-engine/guides/charts_array)

The following examples rely on array and list data generated by reduction of image bands and image metadata using the ee.Reducer.toList() reducer. Note that any set of lists and/or arrays having the same length along a given axis can be plotted by ui.Chart.array.values.

In [ ]:
ndvi_may_19 = ndvi_prueba[4].toArray()
ndvi_may_19.getInfo()

{'bands': [{'crs': 'EPSG:32718',
   'crs_transform': [10, 0, 300000, 0, -10, 8600020],
   'data_type': {'dimensions': 1, 'precision': 'float', 'type': 'PixelType'},
   'dimensions': [10980, 10980],
   'id': 'array'}],
 'type': 'Image'}

In [ ]:
ndviMasked = ndvi_prueba[0].updateMask(ndvi_prueba[0].gte(0.5))
eviMasked  = evi_prueba[0].updateMask(evi_prueba[0].gte(0.5))

dict_prueba = {}
dict_prueba['ndvi_0.5<'] = ndviMasked.clip(roi_clip).getMapId({'min':0, 'max':0.1, 'palette': yuri_ndvi_pal}) # yuri_ndvi_pal, palette1
dict_prueba['evi_0.5<'] = eviMasked.clip(roi_clip).getMapId({'min':0, 'max':0.1, 'palette': yuri_ndvi_pal}) # yuri_ndvi_pal, palette1

mapdisplay(centroide, dict_prueba, zoom_start=11)

### Descarga de Imágenes (Export a Drive)

1. Descarga de un objeto ee.Image
2. Descarga automatizada de un objeto ee.ImageCollection

La API de Python requiere el llamado de `ee.batch` para poder trabajar de manera similar a JavaScript:
- [Exporting data with the Python API](https://developers.google.com/earth-engine/guides/python_install#exporting-data)
- Mas info: [notebook propio para descarga](https://colab.research.google.com/drive/1GCAoapxKzGbT2HcKfKOIf_WDObumoOqJ#scrollTo=mKFt8PFoPj_B)

Si se está usando la API de Earth Engine con Python, no usar las guías de la documentación ya que solo aplican para el code editor:
- [Exporting a collection to Google Drive, Earth Engine Asset, or Google Cloud](https://developers.google.com/earth-engine/tutorials/community/beginners-cookbook#exporting_data)
- [Export.image.toDrive](https://developers.google.com/earth-engine/apidocs/export-image-todrive)

##### Descarga de un objeto `ee.Image`



In [ ]:
#@markdown **Función:** Monitoreo automático de descargas
from ee.batch import Export
from datetime import datetime
import time 

def ee_monitoring(ee_task):
  """
  - Se importa ee.batch.Export
  - Se importa datetime de datetime
  - Se importa time para llamarse cada 5 segundos (.sleep) 
  """
  while ee_task.active():
    print('Sondeo de la tarea (id: {}).'.format(ee_task.id))
    time.sleep(5)
print('ee_monitoring(ee_task) listo!')

ee_monitoring(ee_task) listo!


In [ ]:
#from ee.batch import Export # La función monitoreo automático de descargas ya importa Export
# https://developers.google.com/earth-engine/guides/exporting #exporting-images

task = ee.batch.Export.image.toDrive(image= img_s2_rgb,  # an ee.Image object.
                                     description='prueba1',
                                     scale=10,
                                     folder='Prueba_imagenes')

task.start()
ee_monitoring(task)

Sondeo de la tarea (id: UXC3Z5A42OAI3E2JOLOOLTSW).
Sondeo de la tarea (id: UXC3Z5A42OAI3E2JOLOOLTSW).
Sondeo de la tarea (id: UXC3Z5A42OAI3E2JOLOOLTSW).


Tasks started from the Python API will also appear in the Tasks tab of the JavaScript Code Editor for the same Google account. 
- Revisar el [code editor](https://code.earthengine.google.com/)

In [ ]:
task.status()

{'attempt': 1,
 'creation_timestamp_ms': 1612303866195,
 'description': 'img_ndvi_final',
 'destination_uris': ['https://drive.google.com/#folders/10kptP0blEzkyanKJGuiJ1G4lddoyJTOV'],
 'id': 'WXCXNQBGAXZFIXSZFAMGEVIM',
 'name': 'projects/earthengine-legacy/operations/WXCXNQBGAXZFIXSZFAMGEVIM',
 'start_timestamp_ms': 1612303892573,
 'state': 'COMPLETED',
 'task_type': 'EXPORT_IMAGE',
 'update_timestamp_ms': 1612304046481}

##### Descarga de un objeto `ee.ImageCollection`

###### Descarga automátizada de sentinel-2

In [ ]:
#@markdown *Función Sentinel-2*

def clean_s2(init_date, last_date, roi, cloud_per, calendar, bands):
  def maskS2clouds(image):
    qa = image.select('QA60')
    opaque_cloud = 1 << 10
    cirrus_cloud = 1 << 11
    mask = qa.bitwiseAnd(opaque_cloud).eq(0)\
             .And(qa.bitwiseAnd(cirrus_cloud).eq(0))
    clean_image = image.updateMask(mask).select(bands)
    return clean_image.copyProperties(image, ['system:time_start'])
  ic_s2 = ee.ImageCollection("COPERNICUS/S2_SR")\
               .filterBounds(roi)\
               .filterDate(init_date, last_date)\
               .filter(ee.Filter.calendarRange(calendar[0], calendar[1],'month'))\
               .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', cloud_per))\
               .map(maskS2clouds)
  return ic_s2

In [ ]:
s2 = clean_s2(init_date = '2018-08-12',
              last_date = '2020-09-01',
              roi = roi,
              cloud_per = 50,
              calendar =  [1,12], 
              bands =  ['B4','B3','B2'])
s2.size().getInfo()

51

In [ ]:
ee_get_dates(s2)

###### Descarga automátizada de cualquier image collection

In [ ]:
#@markdown *Función: Descarga Automática a Drive de un objeto* `ee.ImageCollection`
def ee_download_images(ic, region, prefix, folder, scale):
  nimages = ic.size().getInfo()
  ic_list = ic.toList(nimages)    # pasar de un image collection donde cada objeto esta apilado -> al ser una lista, puedo aplicar set o get en GEE
  for n_img in range(nimages):
    to_download = ee.Image(ic_list.get(n_img))
    timestamp = to_download.get('system:time_start').getInfo()
    dt_object = datetime.fromtimestamp(timestamp/1000).strftime('%Y_%m_%d_%H_%M_%S')
    # Create an export task
    task = Export.image.toDrive(
      image=to_download,
      description= prefix+dt_object,
      folder=folder,     
      scale=scale,
      region = region
    )
    print('Descargando: '+prefix+dt_object+'.tif')
    # Start an export task
    task.start()
    ee_monitoring(task)
print('Función: Descarga Automática a Drive listo!')
# el proceso de descarga no es paralelo

Función: Descarga Automática a Drive listo!


In [ ]:
ee_download_images(ic = s2,
                   region =  area_total.geometry().bounds(),
                   prefix = 'CHINCHA_',
                   folder = 'TESIS_S4',
                   scale =  10)

## Fase 2: Random Forest








**Objetivo**
- Realizar la clasificación supervisada mediante el método de ensamble Random Forest con imágenes Sentinel-2.

Pasos:
5. Entrenamiento: Toma de datos en campo y preparación de etiquetado (labels), Clasificador Random Forest (usando EE o [TensorFlow](https://www.tensorflow.org/tutorials/images/classification)/[Scikit-learn](https://kapernikov.com/tutorial-image-classification-with-scikit-learn/))
6. Validación: Evaluación de resultados mediante indicadores de evaluación (Coeficiente de Kappa, Matriz de confusión) 
7. Postprocesamiento de resultados.

[Supervised Classification with EE](https://developers.google.com/earth-engine/guides/classification)

### Árboles de decisión

> **Arboles de decisión**
- Aprendizaje de árboles de decisión es un método comúnmente usado en estadística, data mining y machine learning. Tiene por objetivo la creación de un modelo predictor de valores de la variable objetivo basado en muchas variables de ingreso. [Source](https://en.wikipedia.org/wiki/Decision_tree_learning)
- Presenta un diagrama similar al de un árbol (invertido), donde:
  - cada "nodo interno" denota un test en un atributo
  - cada "rama" representa el resultado de un test y
  - cada "hoja" (nodo terminal) da como respuesta una clase (etiqueta) [Source](https://www.geeksforgeeks.org/decision-tree/)
- Pertenece al tipo de aprendizaje automático supervisado (o supervised machine learning). Es posible construir clasificaciones y regresiones (CART).
- En resumen:
  - Arboles de decisión trabajan en el principio de las **condiciones**
  - El algoritmo revisa las condiciones en cada nodo y "divide" el conjunto de datos, según el resultado del test.
  - Secuencia de preguntas if-else a features individuales
  - objetivo: inferir las etiquetas de clases
  - capaz de capturar relaciones no-lineales entre features y labels
  - no requiere escalamiento (estandarización,...)

<center>
<img src='https://blogs.fu-berlin.de/reseda/files/2018/09/rf_001-768x405.png' width=70%/>
</center>

> **Impureza de Gini**
$$ 
Gini = 1 - \sum \limits_{i=1}^{C}(p^2) 
$$
$$
Gini(Leaf_k) = 1- \sum_{class=1}^{C \, classes} (probability(class|Leaf_k))^2 = Gini(L_k) = 1 - \sum_{c=1}^{C} \left( p(c|L_k) \right )^2
$$




<center>
<img src='https://blogs.fu-berlin.de/reseda/files/2018/09/rrf_002.png' width=90%/>
</center>

> **Ejemplo**\
Para la primera división en nuestro ejemplo que se muestra en la figura anterior, las matemáticas serían:
1. Calcule Gini para el subnodo izquierdo:
$$
Gini(L_1) = 1 - \left(\frac{160}{310} \right)^2 - \left(\frac{40}{310} \right)^2 - \left(\frac{10}{310} \right)^2- \left(\frac{100}{310} \right)^2 = 0.612
$$
2. Calcule Gini para el subnodo derecho:
$$
Gini(L_2) = 1 - \left(\frac{40}{190} \right)^2 - \left(\frac{80}{190} \right)^2 - \left(\frac{70}{190} \right)^2 - \left(\frac{0}{190} \right)^2 = 0.643
$$
3. Calcule el Gini ponderado para dividir (banda 2):
$$
\begin{align}
        Gini(I_1)
        &=
        \sum_{k=1}^{K} 
        \left(\frac{count(L_k)}{count(L_1,...,L_K)}\right)Gini(L_k)\\
        & =
        \left(\frac{count(L_1)}{count(L_1 + L_2)}\right)Gini(L_1)\,+
        \left(\frac{count(L_2)}{count(L_1 + L_2)}\right)Gini(L_2)\\
        & =
        \left(\frac{310}{500} \right)0.612 + \left(\frac{190}{500} \right)0.643
        \\
        & = 0.624
    \end{align}
$$
El algoritmo no usa todas las bandas ($mtry$) para división:
$$
\begin{align}
mtry = \sqrt{M} \\
= \sqrt{11}\\ \approx3
\end{align}
$$

<center>
<img src='https://blogs.fu-berlin.de/reseda/files/2018/09/rf_002-768x486.png' width=80%/>
</center>

### Clasificación Supervisada con Earth Engine

> [**Machine Learning in Earth Engine**](https://developers.google.com/earth-engine/guides/machine-learning)

- Complejidad
    - Para proyectos simples: EE API methods
    - Para proyectos complejos: TensorFlow
- Clasificadores disponibles en Earth Engine [(**ee.Classifier**)](https://developers.google.com/earth-engine/guides/classification):
    - **Classification and Regression Trees (CART)**
    - **Random Forest**
    - NaiveBayes
    - Support Vector Machines (SVM)  

> **Machine Learning en Earth Engine: Metodología**
El modulo **ee.Classifier** maneja la clasificacion supervisada en Earth Engine. El flujo de trabajo general para la clasificacion es:
1. Recopilar datos de entrenamiento. Reuna puntos que tienen como propiedad el **"target"** y las variables predictoras.
2. Instanciar un clasificador. Establezca sus parametros si es necesario.
3. Entrene al clasificador utilizando los datos de entrenamiento.
4. Clasificar una imagen o FeatureCollection.
5. Estime el error de clasificacion con datos de validacion independientes.

###### Imagen a tratar

In [ ]:
# @markdown Función NDVI - EVI

def ndvi_icol(image):
  ndvi = image.expression(
      '(nir - red) / (nir + red)',
      {'red' : image.select('B4').divide(10000),
       'nir' : image.select('B8').divide(10000)}).rename('NDVI')
  return ndvi

def evi_icol(img):
  """
  https://developers.google.com/earth-engine/guides/image_math#expressions
  """
  evi = img.expression('2.5 * (nir - red) / (nir + 6 * red - 7.5 * blue + 1)',
                       {'red' : img.select('B4').divide(10000),
                        'nir' : img.select('B8').divide(10000),
                        'blue': img.select('B2').divide(10000)}).rename('EVI')

  return evi

In [ ]:
img_first = ee.Image('COPERNICUS/S2_SR/20190501T151711_20190501T151806_T18LUL').select(['B4','B3','B2','B8'])

# EVI 
palette = ['FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
           '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
           '012E01', '011D01', '011301']

img_evi = evi_icol(img_first)
img_ndvi = ndvi_icol(img_first)

img_first = img_first.addBands(img_evi, ['EVI'])
img_first = img_first.addBands(img_ndvi, ['NDVI'])

# Diccionario de visualizacion
dict_viz = {'evi':img_evi.getMapId({'min':0.0, 'max':1.0, 'palette': palette}),
            'evi_b&w':img_evi.getMapId({'min':0.0, 'max':1.0}),
            'ndvi':img_ndvi.getMapId({'min':0.0, 'max':1.0, 'palette': palette}),
            'ndvi_b&w':img_ndvi.getMapId({'min':0.0, 'max':1.0}),
            'sentinel2':img_first.multiply(0.0001).getMapId({'min':0.0, 'max':0.3, 'bands':['B4','B3','B2']})}

mapdisplay(centroide, dict_viz, zoom_start=12)

##### Muestreo: Earth Engine Code Editor

> Documentation

- The training data is a `FeatureCollection` with a property storing the class label and properties storing predictor variables. 
- Class labels should be consecutive, integers starting from 0. If necessary, use remap() to convert class values to consecutive integers. 
- The predictors should be numeric.

Training and/or validation data can come from a variety of sources: 
- To collect training data interactively in Earth Engine, you can use the geometry drawing tools (see the geometry tools section of the Code Editor page). 
- Alternatively, you can import predefined training data from an Earth Engine table asset (see the Importing section for details). 

Process:
- Get a classifier from one of the constructors in ee.Classifier. 
- Train the classifier using classifier.train(). 
- Classify an Image or FeatureCollection using classify(). 

Para la recolección de data de entrenamiento y/o validación se pueden usar:
- puntos



```
      agua = ee.FeatureCollection([
              ee.Feature(ee.Geometry.Point([-76.20793238263802, -13.53834059592441]),
                  {"class": 0, "system:index": "0"}),
              ee.Feature(ee.Geometry.Point([-76.19866266828255, -13.58506562431329]),
                  {"class": 0, "system:index": "1"})])
      cultivo = ee.FeatureCollection([
              ee.Feature(ee.Geometry.Point([-76.20793238263802, -13.53834059592441]),
                  {"class": 1, "system:index": "0"}),
              ee.Feature(ee.Geometry.Point([-76.19866266828255, -13.58506562431329]),
                  {"class": 1, "system:index": "1"})])
```

- polígonos: If the training data are polygons representing homogeneous regions, every pixel in each polygon is a training point.

```
        // Manually created polygons.
        var forest1 = ee.Geometry.Rectangle(-63.0187, -9.3958, -62.9793, -9.3443);
        var forest2 = ee.Geometry.Rectangle(-62.8145, -9.206, -62.7688, -9.1735);
        var nonForest1 = ee.Geometry.Rectangle(-62.8161, -9.5001, -62.7921, -9.4486);
        var nonForest2 = ee.Geometry.Rectangle(-62.6788, -9.044, -62.6459, -8.9986);

        // Make a FeatureCollection from the hand-made geometries.
        var polygons = ee.FeatureCollection([
          ee.Feature(nonForest1, {'class': 0}),
          ee.Feature(nonForest2, {'class': 0}),
          ee.Feature(forest1, {'class': 1}),
          ee.Feature(forest2, {'class': 1}),
        ]);

        // Get the values for all pixels in each polygon in the training.
        var training = image.sampleRegions({
          // Get the sample from the polygons FeatureCollection.
          collection: polygons,
          // Keep this list of properties from the polygons.
          properties: ['class'],
          // Set the scale to get Landsat pixels in the polygons.
          scale: 30
        });
```



In [ ]:
# Ejemplo de Training data: Points
points = ee.FeatureCollection('GOOGLE/EE/DEMOS/demo_landcover_labels')
# points.getInfo()

from pprint import pprint
pprint(points.getInfo())

> **Puntos de entrenamiento**: deben almacenarse en un ee.FeatureCollection 

5 clases:
- agua
- cultivo
- urbano
- suelo
- suelo_labrado

In [ ]:
# @markdown 5 clases: 40-`agua`, 50-`cultivo`, 40-`urbano`, 40-`suelo` y 40-`suelo_labrado`
agua = ee.FeatureCollection([
    ee.Feature(ee.Geometry.Point([-76.20793238263802, -13.53834059592441]), {
        "class": 0,
        "system:index": "0"
    }),
    ee.Feature(ee.Geometry.Point([-76.19866266828255, -13.58506562431329]), {
        "class": 0,
        "system:index": "1"
    }),
    ee.Feature(ee.Geometry.Point([-76.20690241437632, -13.49594638116929]), {
        "class": 0,
        "system:index": "2"
    }),
    ee.Feature(ee.Geometry.Point([-76.19780436139779, -13.475247256225725]), {
        "class": 0,
        "system:index": "3"
    }),
    ee.Feature(ee.Geometry.Point([-76.19866266828257, -13.459220896366487]), {
        "class": 0,
        "system:index": "4"
    }),
    ee.Feature(ee.Geometry.Point([-76.20793238263802, -13.457217525920317]), {
        "class": 0,
        "system:index": "5"
    }),
    ee.Feature(ee.Geometry.Point([-76.19608774762825, -13.492774857414556]), {
        "class": 0,
        "system:index": "6"
    }),
    ee.Feature(ee.Geometry.Point([-76.21102228742319, -13.518646583658844]), {
        "class": 0,
        "system:index": "7"
    }),
    ee.Feature(ee.Geometry.Point([-76.19625940900521, -13.558366733495422]), {
        "class": 0,
        "system:index": "8"
    }),
    ee.Feature(ee.Geometry.Point([-76.20899024733426, -13.430223694606841]), {
        "class": 0,
        "system:index": "9"
    }),
    ee.Feature(ee.Geometry.Point([-76.20727363356472, -13.420539411003908]), {
        "class": 0,
        "system:index": "10"
    }),
    ee.Feature(ee.Geometry.Point([-76.20521369704129, -13.4649503232095]), {
        "class": 0,
        "system:index": "11"
    }),
    ee.Feature(ee.Geometry.Point([-76.19388404616238, -13.452763185473371]), {
        "class": 0,
        "system:index": "12"
    }),
    ee.Feature(ee.Geometry.Point([-76.20778861769558, -13.405845269392309]), {
        "class": 0,
        "system:index": "13"
    }),
    ee.Feature(ee.Geometry.Point([-76.21156516798855, -13.422543087942042]), {
        "class": 0,
        "system:index": "14"
    }),
    ee.Feature(ee.Geometry.Point([-76.20864692458035, -13.486651908116594]), {
        "class": 0,
        "system:index": "15"
    }),
    ee.Feature(ee.Geometry.Point([-76.20401206740262, -13.564761271279107]), {
        "class": 0,
        "system:index": "16"
    }),
    ee.Feature(ee.Geometry.Point([-76.2084752632034, -13.551098603686643]), {
        "class": 0,
        "system:index": "17"
    }),
    ee.Feature(ee.Geometry.Point([-76.19954887160183, -13.547093388655492]), {
        "class": 0,
        "system:index": "18"
    }),
    ee.Feature(ee.Geometry.Point([-76.1974889350784, -13.44593940115905]), {
        "class": 0,
        "system:index": "19"
    }),
    ee.Feature(ee.Geometry.Point([-76.21002021559597, -13.397183268045227]), {
        "class": 0,
        "system:index": "20"
    }),
    ee.Feature(ee.Geometry.Point([-76.19645896681668, -13.436923602428658]), {
        "class": 0,
        "system:index": "21"
    }),
    ee.Feature(ee.Geometry.Point([-76.21070686110379, -13.38883362305739]), {
        "class": 0,
        "system:index": "22"
    }),
    ee.Feature(ee.Geometry.Point([-76.2084752632034, -13.449612407120002]), {
        "class": 0,
        "system:index": "23"
    }),
    ee.Feature(ee.Geometry.Point([-76.21002021559597, -13.477325090240653]), {
        "class": 0,
        "system:index": "24"
    }),
    ee.Feature(ee.Geometry.Point([-76.19611564406277, -13.57212487506864]), {
        "class": 0,
        "system:index": "25"
    }),
    ee.Feature(ee.Geometry.Point([-76.2029820991409, -13.510709598293026]), {
        "class": 0,
        "system:index": "26"
    }),
    ee.Feature(ee.Geometry.Point([-76.19508567580105, -13.50436690104268]), {
        "class": 0,
        "system:index": "27"
    }),
    ee.Feature(ee.Geometry.Point([-76.20675864943387, -13.587142500129493]), {
        "class": 0,
        "system:index": "28"
    }),
    ee.Feature(ee.Geometry.Point([-76.19714561232449, -13.52339448660891]), {
        "class": 0,
        "system:index": "29"
    }),
    ee.Feature(ee.Geometry.Point([-76.19954887160183, -13.5347435510398]), {
        "class": 0,
        "system:index": "30"
    }),
    ee.Feature(ee.Geometry.Point([-76.20607200392605, -13.471649259886668]), {
        "class": 0,
        "system:index": "31"
    }),
    ee.Feature(ee.Geometry.Point([-76.20675864943387, -13.440262826676603]), {
        "class": 0,
        "system:index": "32"
    }),
    ee.Feature(ee.Geometry.Point([-76.20160880812527, -13.429243210222896]), {
        "class": 0,
        "system:index": "33"
    }),
    ee.Feature(ee.Geometry.Point([-76.21087852248074, -13.412378878106919]), {
        "class": 0,
        "system:index": "34"
    }),
    ee.Feature(ee.Geometry.Point([-76.20126548537137, -13.41889098830647]), {
        "class": 0,
        "system:index": "35"
    }),
    ee.Feature(ee.Geometry.Point([-76.20092216261746, -13.485003940349847]), {
        "class": 0,
        "system:index": "36"
    }),
    ee.Feature(ee.Geometry.Point([-76.20710197218777, -13.526064903369695]), {
        "class": 0,
        "system:index": "37"
    }),
    ee.Feature(ee.Geometry.Point([-76.21019187697293, -13.501028571594818]), {
        "class": 0,
        "system:index": "38"
    }),
    ee.Feature(ee.Geometry.Point([-76.20813194044949, -13.574794744561816]), {
        "class": 0,
        "system:index": "39"
    })
])

cultivo = ee.FeatureCollection([
    ee.Feature(ee.Geometry.Point([-76.12982645612436, -13.5043623807007]), {
        "class": 1,
        "system:index": "0"
    }),
    ee.Feature(ee.Geometry.Point([-76.1209000645228, -13.50102405118958]), {
        "class": 1,
        "system:index": "1"
    }),
    ee.Feature(ee.Geometry.Point([-76.10021486859995, -13.491509555903567]), {
        "class": 1,
        "system:index": "2"
    }),
    ee.Feature(ee.Geometry.Point([-76.1102570591517, -13.512791454808815]), {
        "class": 1,
        "system:index": "3"
    }),
    ee.Feature(ee.Geometry.Point([-76.13240137677866, -13.513626000391907]), {
        "class": 1,
        "system:index": "4"
    }),
    ee.Feature(ee.Geometry.Point([-76.16218462568003, -13.510120889313345]), {
        "class": 1,
        "system:index": "5"
    }),
    ee.Feature(ee.Geometry.Point([-76.13583460431772, -13.492928408204094]), {
        "class": 1,
        "system:index": "6"
    }),
    ee.Feature(ee.Geometry.Point([-76.16261377912241, -13.482662404377546]), {
        "class": 1,
        "system:index": "7"
    }),
    ee.Feature(ee.Geometry.Point([-76.11411944013315, -13.482161612418878]), {
        "class": 1,
        "system:index": "8"
    }),
    ee.Feature(ee.Geometry.Point([-76.13454714399057, -13.469114939655338]), {
        "class": 1,
        "system:index": "9"
    }),
    ee.Feature(ee.Geometry.Point([-76.15883722882944, -13.46335544251804]), {
        "class": 1,
        "system:index": "10"
    }),
    ee.Feature(ee.Geometry.Point([-76.11154451947885, -13.460350432462805]), {
        "class": 1,
        "system:index": "11"
    }),
    ee.Feature(ee.Geometry.Point([-76.14261522870737, -13.444323075264059]), {
        "class": 1,
        "system:index": "12"
    }),
    ee.Feature(ee.Geometry.Point([-76.11480608564096, -13.443237850864753]), {
        "class": 1,
        "system:index": "13"
    }),
    ee.Feature(ee.Geometry.Point([-76.0992707310267, -13.461435579334665]), {
        "class": 1,
        "system:index": "14"
    }),
    ee.Feature(ee.Geometry.Point([-76.1578930912562, -13.448496969465111]), {
        "class": 1,
        "system:index": "15"
    }),
    ee.Feature(ee.Geometry.Point([-76.17085352521616, -13.474373489856111]), {
        "class": 1,
        "system:index": "16"
    }),
    ee.Feature(ee.Geometry.Point([-76.15634813886362, -13.4724537150875]), {
        "class": 1,
        "system:index": "17"
    }),
    ee.Feature(ee.Geometry.Point([-76.14973917585093, -13.471535556489632]), {
        "class": 1,
        "system:index": "18"
    }),
    ee.Feature(ee.Geometry.Point([-76.11454859357553, -13.448496969465111]), {
        "class": 1,
        "system:index": "19"
    }),
    ee.Feature(ee.Geometry.Point([-76.07772722821909, -13.455341998546404]), {
        "class": 1,
        "system:index": "20"
    }),
    ee.Feature(ee.Geometry.Point([-76.08081713300425, -13.463522386414887]), {
        "class": 1,
        "system:index": "21"
    }),
    ee.Feature(ee.Geometry.Point([-76.11095242701867, -13.433804539541429]), {
        "class": 1,
        "system:index": "22"
    }),
    ee.Feature(ee.Geometry.Point([-76.11091960693547, -13.400394263904234]), {
        "class": 1,
        "system:index": "23"
    }),
    ee.Feature(ee.Geometry.Point([-76.13289226318547, -13.382434316230542]), {
        "class": 1,
        "system:index": "24"
    }),
    ee.Feature(ee.Geometry.Point([-76.15126003051945, -13.371847327100147]), {
        "class": 1,
        "system:index": "25"
    }),
    ee.Feature(ee.Geometry.Point([-76.1594997766132, -13.353290982469337]), {
        "class": 1,
        "system:index": "26"
    }),
    ee.Feature(ee.Geometry.Point([-76.18281984556418, -13.35262289985404]), {
        "class": 1,
        "system:index": "27"
    }),
    ee.Feature(ee.Geometry.Point([-76.16651201475364, -13.358719085163417]), {
        "class": 1,
        "system:index": "28"
    }),
    ee.Feature(ee.Geometry.Point([-76.15303659666282, -13.349282459043662]), {
        "class": 1,
        "system:index": "29"
    }),
    ee.Feature(ee.Geometry.Point([-76.13827371824485, -13.366735751916789]), {
        "class": 1,
        "system:index": "30"
    }),
    ee.Feature(ee.Geometry.Point([-76.1739792846511, -13.351620772464168]), {
        "class": 1,
        "system:index": "31"
    }),
    ee.Feature(ee.Geometry.Point([-76.1439385436843, -13.378860416021276]), {
        "class": 1,
        "system:index": "32"
    }),
    ee.Feature(ee.Geometry.Point([-76.07358189780784, -13.496270914638568]), {
        "class": 1,
        "system:index": "33"
    }),
    ee.Feature(ee.Geometry.Point([-76.05658742148948, -13.493683625626284]), {
        "class": 1,
        "system:index": "34"
    }),
    ee.Feature(ee.Geometry.Point([-76.0634538765676, -13.509123483013797]), {
        "class": 1,
        "system:index": "35"
    }),
    ee.Feature(ee.Geometry.Point([-76.10217192028844, -13.523493821296638]), {
        "class": 1,
        "system:index": "36"
    }),
    ee.Feature(ee.Geometry.Point([-76.08457662915075, -13.516734197916094]), {
        "class": 1,
        "system:index": "37"
    }),
    ee.Feature(ee.Geometry.Point([-76.08981230114782, -13.502630169007137]), {
        "class": 1,
        "system:index": "38"
    }),
    ee.Feature(ee.Geometry.Point([-76.08311750744664, -13.537142840471772]), {
        "class": 1,
        "system:index": "39"
    }),
    ee.Feature(ee.Geometry.Point([-76.08286001538121, -13.547072727838245]), {
        "class": 1,
        "system:index": "40"
    }),
    ee.Feature(ee.Geometry.Point([-76.09856703137243, -13.552663322330673]), {
        "class": 1,
        "system:index": "41"
    }),
    ee.Feature(ee.Geometry.Point([-76.10654928540075, -13.554415571153308]), {
        "class": 1,
        "system:index": "42"
    }),
    ee.Feature(ee.Geometry.Point([-76.12945769438842, -13.554157099375944]), {
        "class": 1,
        "system:index": "43"
    }),
    ee.Feature(ee.Geometry.Point([-76.14009101693098, -13.565263160917079]), {
        "class": 1,
        "system:index": "44"
    }),
    ee.Feature(ee.Geometry.Point([-76.16180618111555, -13.569184634753869]), {
        "class": 1,
        "system:index": "45"
    }),
    ee.Feature(ee.Geometry.Point([-76.16558273140852, -13.564011613080748]), {
        "class": 1,
        "system:index": "46"
    }),
    ee.Feature(ee.Geometry.Point([-76.1341686994261, -13.519729235215385]), {
        "class": 1,
        "system:index": "47"
    }),
    ee.Feature(ee.Geometry.Point([-76.15257938210432, -13.529743297778847]), {
        "class": 1,
        "system:index": "48"
    }),
    ee.Feature(ee.Geometry.Point([-76.15596969429915, -13.521690355614806]), {
        "class": 1,
        "system:index": "49"
    })
])

urbano = ee.FeatureCollection([
    ee.Feature(ee.Geometry.Point([-76.13196117411108, -13.426720730582748]), {
        "class": 2,
        "system:index": "0"
    }),
    ee.Feature(ee.Geometry.Point([-76.13683206568213, -13.426470276238721]), {
        "class": 2,
        "system:index": "1"
    }),
    ee.Feature(ee.Geometry.Point([-76.13912803659888, -13.422546457397523]), {
        "class": 2,
        "system:index": "2"
    }),
    ee.Feature(ee.Geometry.Point([-76.12985832224341, -13.420939342869776]), {
        "class": 2,
        "system:index": "3"
    }),
    ee.Feature(ee.Geometry.Point([-76.12792713175268, -13.428620000852026]), {
        "class": 2,
        "system:index": "4"
    }),
    ee.Feature(ee.Geometry.Point([-76.13633853922339, -13.429037420874405]), {
        "class": 2,
        "system:index": "5"
    }),
    ee.Feature(ee.Geometry.Point([-76.1403940392539, -13.425865010495018]), {
        "class": 2,
        "system:index": "6"
    }),
    ee.Feature(ee.Geometry.Point([-76.14148838053198, -13.428348677448108]), {
        "class": 2,
        "system:index": "7"
    }),
    ee.Feature(ee.Geometry.Point([-76.13884908686133, -13.428703484914584]), {
        "class": 2,
        "system:index": "8"
    }),
    ee.Feature(ee.Geometry.Point([-76.13374216089697, -13.422191640829714]), {
        "class": 2,
        "system:index": "9"
    }),
    ee.Feature(ee.Geometry.Point([-76.13348466883154, -13.432188918605823]), {
        "class": 2,
        "system:index": "10"
    }),
    ee.Feature(ee.Geometry.Point([-76.13533002863379, -13.416117934750881]), {
        "class": 2,
        "system:index": "11"
    }),
    ee.Feature(ee.Geometry.Point([-76.13022310266943, -13.415324793408919]), {
        "class": 2,
        "system:index": "12"
    }),
    ee.Feature(ee.Geometry.Point([-76.12803442011328, -13.41774595351081]), {
        "class": 2,
        "system:index": "13"
    }),
    ee.Feature(ee.Geometry.Point([-76.13962156305762, -13.416201423160851]), {
        "class": 2,
        "system:index": "14"
    }),
    ee.Feature(ee.Geometry.Point([-76.13704664240332, -13.436279542658044]), {
        "class": 2,
        "system:index": "15"
    }),
    ee.Feature(ee.Geometry.Point([-76.13275510797949, -13.434442944570007]), {
        "class": 2,
        "system:index": "16"
    }),
    ee.Feature(ee.Geometry.Point([-76.1279485894248, -13.424174432574109]), {
        "class": 2,
        "system:index": "17"
    }),
    ee.Feature(ee.Geometry.Point([-76.11687643061133, -13.410983341774543]), {
        "class": 2,
        "system:index": "18"
    }),
    ee.Feature(ee.Geometry.Point([-76.12365705500098, -13.40255068275863]), {
        "class": 2,
        "system:index": "19"
    }),
    ee.Feature(ee.Geometry.Point([-76.14108068476172, -13.41081636129894]), {
        "class": 2,
        "system:index": "20"
    }),
    ee.Feature(ee.Geometry.Point([-76.0847757531211, -13.438783971930006]), {
        "class": 2,
        "system:index": "21"
    }),
    ee.Feature(ee.Geometry.Point([-76.08168584833594, -13.441622293483208]), {
        "class": 2,
        "system:index": "22"
    }),
    ee.Feature(ee.Geometry.Point([-76.08992559442969, -13.434109016134004]), {
        "class": 2,
        "system:index": "23"
    }),
    ee.Feature(ee.Geometry.Point([-76.14035112390967, -13.438825712196282]), {
        "class": 2,
        "system:index": "24"
    }),
    ee.Feature(ee.Geometry.Point([-76.1321972085044, -13.438533530179818]), {
        "class": 2,
        "system:index": "25"
    }),
    ee.Feature(ee.Geometry.Point([-76.1616800499961, -13.426178079173708]), {
        "class": 2,
        "system:index": "26"
    }),
    ee.Feature(ee.Geometry.Point([-76.16588575373144, -13.427305123035735]), {
        "class": 2,
        "system:index": "27"
    }),
    ee.Feature(ee.Geometry.Point([-76.15790349970312, -13.398626473265448]), {
        "class": 2,
        "system:index": "28"
    }),
    ee.Feature(ee.Geometry.Point([-76.16030675898047, -13.408478622457748]), {
        "class": 2,
        "system:index": "29"
    }),
    ee.Feature(ee.Geometry.Point([-76.14348394403906, -13.391946820518598]), {
        "class": 2,
        "system:index": "30"
    }),
    ee.Feature(ee.Geometry.Point([-76.13610250483008, -13.395954634421196]), {
        "class": 2,
        "system:index": "31"
    }),
    ee.Feature(ee.Geometry.Point([-76.13764745722266, -13.400296357472063]), {
        "class": 2,
        "system:index": "32"
    }),
    ee.Feature(ee.Geometry.Point([-76.12992269525976, -13.397624537176565]), {
        "class": 2,
        "system:index": "33"
    }),
    ee.Feature(ee.Geometry.Point([-76.12734777460547, -13.407309744509991]), {
        "class": 2,
        "system:index": "34"
    }),
    ee.Feature(ee.Geometry.Point([-76.12236959467383, -13.413988970492584]), {
        "class": 2,
        "system:index": "35"
    }),
    ee.Feature(ee.Geometry.Point([-76.12202627191992, -13.417996416955955]), {
        "class": 2,
        "system:index": "36"
    }),
    ee.Feature(ee.Geometry.Point([-76.14708883295508, -13.41482386068576]), {
        "class": 2,
        "system:index": "37"
    }),
    ee.Feature(ee.Geometry.Point([-76.1513803673789, -13.400296357472063]), {
        "class": 2,
        "system:index": "38"
    }),
    ee.Feature(ee.Geometry.Point([-76.15277528754547, -13.415256617545504]), {
        "class": 2,
        "system:index": "39"
    })
])

suelo = ee.FeatureCollection([
    ee.Feature(ee.Geometry.Point([-76.13045930854156, -13.353132987581796]), {
        "class": 3,
        "system:index": "0"
    }),
    ee.Feature(ee.Geometry.Point([-76.11294984809234, -13.349458507912184]), {
        "class": 3,
        "system:index": "1"
    }),
    ee.Feature(ee.Geometry.Point([-76.09132051459625, -13.376514728873774]), {
        "class": 3,
        "system:index": "2"
    }),
    ee.Feature(ee.Geometry.Point([-76.09269380561187, -13.395218488527172]), {
        "class": 3,
        "system:index": "3"
    }),
    ee.Feature(ee.Geometry.Point([-76.07724428168609, -13.402232023656712]), {
        "class": 3,
        "system:index": "4"
    }),
    ee.Feature(ee.Geometry.Point([-76.05286836615875, -13.436294854167137]), {
        "class": 3,
        "system:index": "5"
    }),
    ee.Feature(ee.Geometry.Point([-76.0449719428189, -13.418262192799528]), {
        "class": 3,
        "system:index": "6"
    }),
    ee.Feature(ee.Geometry.Point([-76.02505922309234, -13.421267730440047]), {
        "class": 3,
        "system:index": "7"
    }),
    ee.Feature(ee.Geometry.Point([-76.00686311713531, -13.44163760465113]), {
        "class": 3,
        "system:index": "8"
    }),
    ee.Feature(ee.Geometry.Point([-76.0010266303189, -13.480035117633477]), {
        "class": 3,
        "system:index": "9"
    }),
    ee.Feature(ee.Geometry.Point([-75.97802400580719, -13.464676851615506]), {
        "class": 3,
        "system:index": "10"
    }),
    ee.Feature(ee.Geometry.Point([-76.02471590033844, -13.572831425053622]), {
        "class": 3,
        "system:index": "11"
    }),
    ee.Feature(ee.Geometry.Point([-75.98832368842437, -13.503404498312044]), {
        "class": 3,
        "system:index": "12"
    }),
    ee.Feature(ee.Geometry.Point([-76.02059602729156, -13.49839696643072]), {
        "class": 3,
        "system:index": "13"
    }),
    ee.Feature(ee.Geometry.Point([-76.03535890570953, -13.509079574060449]), {
        "class": 3,
        "system:index": "14"
    }),
    ee.Feature(ee.Geometry.Point([-76.04256868354156, -13.535116424860643]), {
        "class": 3,
        "system:index": "15"
    }),
    ee.Feature(ee.Geometry.Point([-76.01132631293609, -13.519427893914546]), {
        "class": 3,
        "system:index": "16"
    }),
    ee.Feature(ee.Geometry.Point([-76.05527162543609, -13.555810351863943]), {
        "class": 3,
        "system:index": "17"
    }),
    ee.Feature(ee.Geometry.Point([-76.03707551947906, -13.552806506458701]), {
        "class": 3,
        "system:index": "18"
    }),
    ee.Feature(ee.Geometry.Point([-76.03089570990875, -13.518092652080412]), {
        "class": 3,
        "system:index": "19"
    }),
    ee.Feature(ee.Geometry.Point([-76.02952241889312, -13.536117785356874]), {
        "class": 3,
        "system:index": "20"
    }),
    ee.Feature(ee.Geometry.Point([-75.97768068305328, -13.533113691229662]), {
        "class": 3,
        "system:index": "21"
    }),
    ee.Feature(ee.Geometry.Point([-75.98729372016265, -13.52510258823432]), {
        "class": 3,
        "system:index": "22"
    }),
    ee.Feature(ee.Geometry.Point([-75.97047090522125, -13.482038296995597]), {
        "class": 3,
        "system:index": "23"
    }),
    ee.Feature(ee.Geometry.Point([-76.00136995307281, -13.42226956796032]), {
        "class": 3,
        "system:index": "24"
    }),
    ee.Feature(ee.Geometry.Point([-75.9951901435025, -13.403233940550326]), {
        "class": 3,
        "system:index": "25"
    }),
    ee.Feature(ee.Geometry.Point([-76.01372957221344, -13.40724156637249]), {
        "class": 3,
        "system:index": "26"
    }),
    ee.Feature(ee.Geometry.Point([-76.04840517035797, -13.394884505563192]), {
        "class": 3,
        "system:index": "27"
    }),
    ee.Feature(ee.Geometry.Point([-76.07484102240875, -13.382860810175435]), {
        "class": 3,
        "system:index": "28"
    }),
    ee.Feature(ee.Geometry.Point([-76.06454133979156, -13.360481779100406]), {
        "class": 3,
        "system:index": "29"
    }),
    ee.Feature(ee.Geometry.Point([-76.11157655707672, -13.365158166248213]), {
        "class": 3,
        "system:index": "30"
    }),
    ee.Feature(ee.Geometry.Point([-76.08754396430328, -13.355805301300215]), {
        "class": 3,
        "system:index": "31"
    }),
    ee.Feature(ee.Geometry.Point([-76.18882417670562, -13.385198797978665]), {
        "class": 3,
        "system:index": "32"
    }),
    ee.Feature(ee.Geometry.Point([-76.19946718207672, -13.379186783483458]), {
        "class": 3,
        "system:index": "33"
    }),
    ee.Feature(ee.Geometry.Point([-76.20908021918609, -13.357141447063093]), {
        "class": 3,
        "system:index": "34"
    }),
    ee.Feature(ee.Geometry.Point([-76.04565858832672, -13.511416330709027]), {
        "class": 3,
        "system:index": "35"
    }),
    ee.Feature(ee.Geometry.Point([-75.99965333930328, -13.538788059414708]), {
        "class": 3,
        "system:index": "36"
    }),
    ee.Feature(ee.Geometry.Point([-76.20152711860015, -13.343445602381413]), {
        "class": 3,
        "system:index": "37"
    }),
    ee.Feature(ee.Geometry.Point([-75.97939729682281, -13.417260338560412]), {
        "class": 3,
        "system:index": "38"
    }),
    ee.Feature(ee.Geometry.Point([-76.02402925483062, -13.368498387264628]), {
        "class": 3,
        "system:index": "39"
    })
])

suelo_labrado = ee.FeatureCollection([
    ee.Feature(ee.Geometry.Point([-76.12129161206124, -13.518338550816555]), {
        "class": 4,
        "system:index": "0"
    }),
    ee.Feature(ee.Geometry.Point([-76.09472701397775, -13.511912601499546]), {
        "class": 4,
        "system:index": "1"
    }),
    ee.Feature(ee.Geometry.Point([-76.07411210318345, -13.513316217291814]), {
        "class": 4,
        "system:index": "2"
    }),
    ee.Feature(ee.Geometry.Point([-76.08767335196275, -13.530131709926529]), {
        "class": 4,
        "system:index": "3"
    }),
    ee.Feature(ee.Geometry.Point([-76.11453835745591, -13.541313563823314]), {
        "class": 4,
        "system:index": "4"
    }),
    ee.Feature(ee.Geometry.Point([-76.12762753744859, -13.540187057566474]), {
        "class": 4,
        "system:index": "5"
    }),
    ee.Feature(ee.Geometry.Point([-76.146252796848, -13.555457021641763]), {
        "class": 4,
        "system:index": "6"
    }),
    ee.Feature(ee.Geometry.Point([-76.1175424315526, -13.565636452681545]), {
        "class": 4,
        "system:index": "7"
    }),
    ee.Feature(ee.Geometry.Point([-76.13058869620103, -13.568681853406785]), {
        "class": 4,
        "system:index": "8"
    }),
    ee.Feature(ee.Geometry.Point([-76.15595166464585, -13.563675694558972]), {
        "class": 4,
        "system:index": "9"
    }),
    ee.Feature(ee.Geometry.Point([-76.06257025307376, -13.526206914909006]), {
        "class": 4,
        "system:index": "10"
    }),
    ee.Feature(ee.Geometry.Point([-76.07116438990379, -13.514607184706003]), {
        "class": 4,
        "system:index": "11"
    }),
    ee.Feature(ee.Geometry.Point([-76.06765844881456, -13.507190952511266]), {
        "class": 4,
        "system:index": "12"
    }),
    ee.Feature(ee.Geometry.Point([-76.06249913739491, -13.496979616338484]), {
        "class": 4,
        "system:index": "13"
    }),
    ee.Feature(ee.Geometry.Point([-76.03932485150624, -13.483604738981146]), {
        "class": 4,
        "system:index": "14"
    }),
    ee.Feature(ee.Geometry.Point([-76.05001077222157, -13.486359065131227]), {
        "class": 4,
        "system:index": "15"
    }),
    ee.Feature(ee.Geometry.Point([-76.04820832776356, -13.475383303959685]), {
        "class": 4,
        "system:index": "16"
    }),
    ee.Feature(ee.Geometry.Point([-76.06962308453846, -13.469039668493302]), {
        "class": 4,
        "system:index": "17"
    }),
    ee.Feature(ee.Geometry.Point([-76.08005151318837, -13.476802252020589]), {
        "class": 4,
        "system:index": "18"
    }),
    ee.Feature(ee.Geometry.Point([-76.03120893147721, -13.462797812654347]), {
        "class": 4,
        "system:index": "19"
    }),
    ee.Feature(ee.Geometry.Point([-76.06564977334654, -13.456139571341474]), {
        "class": 4,
        "system:index": "20"
    }),
    ee.Feature(ee.Geometry.Point([-76.08105638192808, -13.452591879086084]), {
        "class": 4,
        "system:index": "21"
    }),
    ee.Feature(ee.Geometry.Point([-76.12101056741392, -13.456849103485833]), {
        "class": 4,
        "system:index": "22"
    }),
    ee.Feature(ee.Geometry.Point([-76.09998204873716, -13.445245902210852]), {
        "class": 4,
        "system:index": "23"
    }),
    ee.Feature(ee.Geometry.Point([-76.17044904397642, -13.457934266228115]), {
        "class": 4,
        "system:index": "24"
    }),
    ee.Feature(ee.Geometry.Point([-76.14839055703794, -13.444411117861641]), {
        "class": 4,
        "system:index": "25"
    }),
    ee.Feature(ee.Geometry.Point([-76.10993840860044, -13.424598952231626]), {
        "class": 4,
        "system:index": "26"
    }),
    ee.Feature(ee.Geometry.Point([-76.17765882180845, -13.39838308231166]), {
        "class": 4,
        "system:index": "27"
    }),
    ee.Feature(ee.Geometry.Point([-76.18120916204748, -13.35827434374201]), {
        "class": 4,
        "system:index": "28"
    }),
    ee.Feature(ee.Geometry.Point([-76.18901975469885, -13.343826943139966]), {
        "class": 4,
        "system:index": "29"
    }),
    ee.Feature(ee.Geometry.Point([-76.08138807134924, -13.426173056836728]), {
        "class": 4,
        "system:index": "30"
    }),
    ee.Feature(ee.Geometry.Point([-76.1276976734633, -13.493285539193698]), {
        "class": 4,
        "system:index": "31"
    }),
    ee.Feature(ee.Geometry.Point([-76.15456267895647, -13.500045827184273]), {
        "class": 4,
        "system:index": "32"
    }),
    ee.Feature(ee.Geometry.Point([-76.1793677479262, -13.51147945863916]), {
        "class": 4,
        "system:index": "33"
    }),
    ee.Feature(ee.Geometry.Point([-76.169068065309, -13.487276233626135]), {
        "class": 4,
        "system:index": "34"
    }),
    ee.Feature(ee.Geometry.Point([-76.16005584301897, -13.524748460318634]), {
        "class": 4,
        "system:index": "35"
    }),
    ee.Feature(ee.Geometry.Point([-76.17318793835588, -13.540353195681673]), {
        "class": 4,
        "system:index": "36"
    }),
    ee.Feature(ee.Geometry.Point([-76.16554900708147, -13.557876009999442]), {
        "class": 4,
        "system:index": "37"
    }),
    ee.Feature(ee.Geometry.Point([-76.09173083704663, -13.518776163711808]), {
        "class": 4,
        "system:index": "38"
    }),
    ee.Feature(ee.Geometry.Point([-76.14631915491772, -13.540222518667973]), {
        "class": 4,
        "system:index": "39"
    })
])

In [ ]:
# Merge the three geometry layers into a single FeatureCollection.
classes = agua.merge(cultivo).merge(urbano).merge(suelo).merge(suelo_labrado)
classes.size().getInfo()

210

In [ ]:
# Bandas de la imagen
img = img_first.clip(roi_clip)
print('Bandas:', img.bandNames().getInfo())

# visualizando datos de entrada
center = classes.geometry().centroid().coordinates().getInfo()

mapdisplay(center,{'train_dataset':classes.getInfo(),
                   'img':img.divide(10000).getMapId({'min':0.0, 'max':0.3, 'bands':['B4','B3','B2']})}, zoom_start=12)

Bandas: ['B4', 'B3', 'B2', 'B8', 'EVI', 'NDVI']


##### Entrenamiento: Áboles de decisión (CART)

Deprecated classifiers: https://groups.google.com/g/google-earthengine-announce/c/rCu4FP_Cn08/m/DqC192X9BAAJ

In [ ]:
# Bandas para prediccion
bands = img.bandNames().getInfo()

# Esta propiedad de la tabla almacena el target (cobertura del suelo).
label = 'class' # OJO: puedes poner landcover si en el editor pusiste de nombre landcover en lugar de class

# Superponga los puntos en las imagenes para obtener los datos de entrenamiento.
training = img.select(bands).sampleRegions(
  collection = classes,
  properties = [label],
  scale = 10)

# Entrena un clasificador CART con parámetros predeterminados.
trained = ee.Classifier.smileCart().train(training, label, bands)

# Clasifique la imagen con las mismas bandas utilizadas para el entrenamiento.
classified = img.select(bands).classify(trained)

mapdisplay(center,{
    # 'train_dataset': classes.getInfo(),
                   'image':img.divide(10000).getMapId({'min':0.0, 'max':0.3,'bands':['B4','B3','B2']}),
                   'clasificacion':classified.getMapId({'min': 0, 'max': 4, 'palette': ['#0c2bd6', '#3b8b00', '#d63000', '#c5b03a','#1de6ff']})}, zoom_start=12)

In [ ]:
cart_result = mapViz
cart_result

In [ ]:
train_accuracy = trained.confusionMatrix()

print('Validation error matrix    :', train_accuracy.getInfo())
print('Validation overall accuracy:', train_accuracy.accuracy().getInfo())

Validation error matrix    : [[40, 0, 0, 0, 0], [0, 50, 0, 0, 0], [0, 0, 40, 0, 0], [0, 0, 0, 40, 0], [0, 0, 0, 0, 40]]
Validation overall accuracy: 1


##### Entrenamiento: Random Forest

> **Ensemble Learning**
- Train different models on the same dataset
- Let each model make its predictions
- Meta-model: aggregates predictions of individual models
- FInal prediction: more robust and less prone to errors
- Best results: models are skillful in different ways
<center>
<img src='https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Fcdn-images-1.medium.com%2Fmax%2F1200%2F1*10t9S7xvWE5Z3NEZrmHG2w.jpeg&f=1&nofb=1' width=70%/>
</center>

<center>
<img src='https://i0.wp.com/dimensionless.in/wp-content/uploads/RandomForest_blog_files/figure-html/voting.png?w=1080&ssl=1' align='center' width=100%/>
</center>

In [ ]:
training = img.sampleRegions(
  collection = classes,  
  properties = [label],  
  scale = 10
)

# Crear un clasificador RF con parámetros personalizados.
classifier = ee.Classifier.smileRandomForest(25)

# Entrenar el clasificador
trained = classifier.train(training, 'class', bands)

# Clasificar la imagen
classified = img.classify(trained)

# Visualización de clasificación
mapdisplay(center,{
    # 'train_dataset': classes.getInfo(),
                   'image':img.divide(10000).getMapId({'min':0.0, 'max':0.3,'bands':['B4','B3','B2']}),
                   'clasificacion':classified.getMapId({'min': 0, 'max': 4, 'palette': ['#0c2bd6', '#0bff17', '#d63000', '#c5b03a','#1de6ff']})}, zoom_start=12)

In [ ]:
result_rf = mapViz
result_rf

In [ ]:
train_accuracy = training.classify(trained)

# Obtenga una matriz de confusión que represente la precisión esperada.
trainAccuracy = train_accuracy.errorMatrix('class', 'classification')
print('Validation error matrix    :', trainAccuracy.getInfo())
print('Validation overall accuracy:', trainAccuracy.accuracy().getInfo())

Validation error matrix    : [[40, 0, 0, 0, 0], [0, 50, 0, 0, 0], [0, 0, 38, 1, 1], [0, 0, 0, 40, 0], [0, 1, 0, 1, 38]]
Validation overall accuracy: 0.9809523809523809


# Referencias - Earth Engine

References:
- [Noel Gorelick: How can Google Earth Engine solve large-scale societal problems?](https://www.youtube.com/watch?v=bXN-0DwGEd0&t=26s)
- [Google Earth Engine: Planetary-scale geospatial analysis for everyone](https://doi.org/10.1016/j.rse.2017.06.031)
- [Google Earth Engine Tutorial: Tensorflow Demo: using Colab & Code Editor](https://www.youtube.com/watch?v=V9bX_sD9uL8)
- [Google Earth Engine GGE Teledetección | MasterLive (Cesar Aybar)](https://www.youtube.com/watch?v=M7WF83ZsINA&t=846s)
- [Tyler A. Erickson - Analyzing Petabytes of Earth Science Data with Jupyter and Earth Engine](https://www.youtube.com/watch?v=LzxQH0Ze0iI)
- [Noel Gorelick: How can Google Earth Engine solve large-scale societal problems?](https://www.youtube.com/watch?v=bXN-0DwGEd0)

Earth Engine Guides:
- [A Quick Introduction to Google Earth Engine - Towards Datascience](https://towardsdatascience.com/a-quick-introduction-to-google-earth-engine-c6a608c5febe)
- [Rapid Classification of Croplands](https://developers.google.com/earth-engine/tutorials/community/classify-maizeland-ng)
- [API Reference / JavaScript - Python Client Libraries](https://developers.google.com/earth-engine/apidocs#client-libraries)
- [JavaScript - Python Syntax](https://developers.google.com/earth-engine/guides/python_install#syntax)
- [Sentinel-2 Cloud Masking with s2cloudless](https://developers.google.com/earth-engine/tutorials/community/sentinel-2-s2cloudless)




Probar:

- Hough transform: https://en.wikipedia.org/wiki/Hough_transform
- Canny edge detector: https://en.wikipedia.org/wiki/Canny_edge_detector

- [Iterating over an ImageCollection](https://developers.google.com/earth-engine/guides/ic_iterating): Warning: Use `getInfo()` with caution!
> You shouldn't use `getInfo()` unless you absolutely need to. If you call `getInfo()` in your code, ***Earth Engine will open the container and tell you what's inside, but it will block the rest of your code until that's done.*** (You can optionally provide a callback function to avoid that, but the better option is to do everything on the server, if possible.)

> [4 Machine Learning Concepts I Wish I Knew When I Built My First Model](https://towardsdatascience.com/4-machine-learning-concepts-i-wish-i-knew-when-i-built-my-first-model-3b8ca9506451)

1. **Model Deployment w/ Simple Web UI's**

Menciona usar [Gradio](https://www.gradio.app/) el cual es un paquete de python que ayuda a desarrollar una aplicación web para tu modelo de ML con pocas lineas de codigo (similar a Streamlit o Flask). 
- permite validacion de modelo
- facil de conducir demos
- facil de distribuir e implementar (no conviene enseñar jupyter notebooks a los demás)

2. **Feature Importance**

Asignar puntajes a las variables de entrada del modelo para saber que tan bueno son para predecir la variable objetivo. Mayor el puntaje, mayor importancia tiene en el modelo (otras variables pueden inducir ruido y reducir la precisión del modelo).

Use feature importance to improve feature selection, model interpretability, and communication. [Ejemplo aquí](https://towardsdatascience.com/a-machine-learning-project-predicting-used-car-prices-efbc4d2a4998)

3. **Hyperparameter Tuning [otro articulo aquí](https://medium.com/mlait/hyperparameters-in-machine-learning-fa45ccec9f6c)**

La escencia de ML es encontrar los parametros que mejor se ajustan a los datos, obtenido mediante entrenamiento del modelo.

Hyperparameters: parametros que no pueden ser aprendidos directamente del proceso de entrenamiento del modelo. Son de conceptos de alto nivel que generalmente son establecidos antes del entrenamiento del modelo.

Ejemplos:
- learning rate
- Numero de hojas (leaves) o profundida maxima (max depth) que un arbol puede tener
- Numero de hidden layers en una red neuronal

Los hiperparametros no son determinados por la data en si mismo, pero los correctos hiperparametros pueden producir una mejora en precisión del modelo a un 80% a más.

Hay tecnicas existentes disponibles para optimizar automaticamente los hiperparametros de ciertos modelos (para que no tengas que testear diferentes opciones/numeros):
- Grid Search
- Random Search 

[Demostración](https://machinelearningmastery.com/hyperparameter-optimization-with-random-search-and-grid-search/)

4. **Evaluación de metricas del modelo**

Knowing what metric(s) to evaluate your machine learning model on ultimately requires you to have a solid understanding of the business problem that you are trying to solve.

Se puede dividir en dos partes:
- Entendimiento de los requerimientos del problema
- Escogiendo las metricas correctas para evaluar el modelo (comparar falsos positivos vs falsos negativos). Nuestra decision final depende en como el modelo va a ser integrado en el proceso (empresarial / final).

Understand the business problem. Have a solid understanding of all relevant metrics, and understand the consequences of choosing each metric.






- [Medium: Building Intuition for Random Forests](https://medium.com/x8-the-ai-community/building-intuition-for-random-forests-76d36fa28c5e)
- [Descriptive Statistics with Python](https://medium.com/dataseries/descriptive-statistics-with-python-75e2b1249e8d)

- [Predicting Deforestation: Acquiring Satellite Imagery](https://medium.com/project-canopy/predicting-deforestation-acquiring-satellite-imagery-82dec0597ea2)
- [Area Monitoring — Expert Judgement Application](https://medium.com/sentinel-hub/expert-judgement-application-67a07f2feac4)

In [ ]:
def ee_get_dates(ic):
  """
  Permite obtener información de la Fecha y Hora de toma de imágen por el satélite.
  - ic: `ee.ImageCollection()`
  """
  dates = ic.aggregate_array('system:time_start').getInfo()
  anonym = lambda x: 
  return list(map(anonym, dates))
print('ee_get_dates(ic) listo!')